<a href="https://colab.research.google.com/github/royalgillz/AGI-Traffic-Agent/blob/main/traffic_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
dtrnngc_ua_detrac_dataset_path = kagglehub.dataset_download('dtrnngc/ua-detrac-dataset')
raguhudarare_videotraffic_path = kagglehub.dataset_download('raguhudarare/videotraffic')

print('Data source import complete.')


100%|██████████| 9.23G/9.23G [01:23<00:00, 119MB/s]

Extracting files...


100%|██████████| 255M/255M [00:01<00:00, 139MB/s]

Extracting files...


Data source import complete.


In [ ]:
# **Step 1: Setup and Unzip Archive Folder**

In [ ]:
import os

# Define paths for Kaggle input dataset
base_path = '/root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload'

train_image_path = os.path.join(base_path, 'images/train')
val_image_path = os.path.join(base_path, 'images/val')

train_label_path = os.path.join(base_path, 'labels/train')
val_label_path = os.path.join(base_path, 'labels/val')

# Print paths to verify
print(f"Train images path: {train_image_path}")
print(f"Validation images path: {val_image_path}")
print(f"Train labels path: {train_label_path}")
print(f"Validation labels path: {val_label_path}")


Train images path: /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/images/train
Validation images path: /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/images/val
Train labels path: /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/labels/train
Validation labels path: /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/labels/val


In [ ]:
print("UA-DETRAC dataset path:", dtrnngc_ua_detrac_dataset_path)
print("VideoTraffic dataset path:", raguhudarare_videotraffic_path)


UA-DETRAC dataset path: /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1
VideoTraffic dataset path: /root/.cache/kagglehub/datasets/raguhudarare/videotraffic/versions/2


# **Step 2: Preprocessing with Progress Prints**

In [ ]:
from PIL import Image
import os

# Convert images to JPG if necessary and provide progress feedback
def convert_images_to_jpg(folder_path):
    total_files = len(os.listdir(folder_path))
    for idx, filename in enumerate(os.listdir(folder_path)):
        if not filename.endswith('.jpg'):
            img = Image.open(os.path.join(folder_path, filename))
            img.convert('RGB').save(os.path.join(folder_path, os.path.splitext(filename)[0] + '.jpg'), 'JPEG')
            os.remove(os.path.join(folder_path, filename))  # Remove old file
        if idx % 5000 == 0:  # Print progress every 5000 images
            print(f"Processed {idx}/{total_files} images in {folder_path}")

print("Converting train images...")
convert_images_to_jpg(train_image_path)

print("Converting validation images...")
convert_images_to_jpg(val_image_path)

# Verify annotations format with progress prints
def check_labels(label_folder):
    total_labels = len(os.listdir(label_folder))
    for idx, label_file in enumerate(os.listdir(label_folder)):
        with open(os.path.join(label_folder, label_file), 'r') as file:
            for line in file.readlines():
                values = line.strip().split(' ')
                assert len(values) == 5, f"Incorrect label format in {label_file}"
        if idx % 5000 == 0:  # Print progress every 5000 labels
            print(f"Verified {idx}/{total_labels} label files in {label_folder}")

print("Verifying train labels...")
check_labels(train_label_path)

print("Verifying validation labels...")
check_labels(val_label_path)

print("Data preprocessing complete!")


Converting train images...
Processed 0/83791 images in /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/images/train
Processed 5000/83791 images in /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/images/train
Processed 10000/83791 images in /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/images/train
Processed 15000/83791 images in /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/images/train
Processed 20000/83791 images in /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/images/train
Processed 25000/83791 images in /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/images/train
Processed 30000/83791 images in /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/c

# **Step 3: Install YOLOv5**

In [ ]:
# Install YOLOv5 dependencies
!git clone https://github.com/ultralytics/yolov5  # Clone YOLOv5 repository
!pip install -r requirements.txt  # Install required Python packages

# If you encounter issues, you might need to install additional packages
!pip install torch torchvision

Cloning into 'yolov5'...
remote: Enumerating objects: 17360, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 17360 (delta 32), reused 13 (delta 10), pack-reused 17308 (from 4)
Receiving objects: 100% (17360/17360), 16.26 MiB | 8.02 MiB/s, done.
Resolving deltas: 100% (11896/11896), done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s e

In [ ]:
%cd yolov5

/content/yolov5


# **Step 4: Training YOLOv5**

In [ ]:
pwd

'/content/yolov5'

In [ ]:
# Create the folder named 'Detrac'
!mkdir /kaggle/working/Detrac_File
# Create the detrac.yaml file with the required structure
detrac_yaml_content = """
train: /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/images/train
val: /root/.cache/kagglehub/datasets/dtrnngc/ua-detrac-dataset/versions/1/content/UA-DETRAC/DETRAC_Upload/images/val
nc: 4 # Number of classes
names:
  0: "class0"
  1: "class1"
  2: "class2"
  3: "class3"
"""

# Write the content to a new detrac.yaml file in the Detrac folder
with open('/kaggle/working/Detrac_File/detrac.yaml', 'w') as f:
    f.write(detrac_yaml_content)

print("detrac.yaml created successfully!")


mkdir: cannot create directory ‘/kaggle/working/Detrac_File’: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/Detrac_File/detrac.yaml'

In [ ]:
!python train.py --img 416 --batch 16 --epochs 10 --data /kaggle/working/Detrac_File/detrac.yaml --weights yolov5s.pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.0/950.0 kB 36.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-03-29 18:34:03.535673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743273243.795922    7039 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743273243.858192    7039 cuda_blas.cc:1418] Unabl

#  **5. Evaluate the Model**

In [ ]:
!python val.py --weights /kaggle/working/yolov5/runs/train/exp3/weights/best.pt --data /kaggle/working/Detrac_File/detrac.yaml --img 416

# **6. Inference on Videos**

In [ ]:
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time  # Added for FPS calculation

# Ensure matplotlib works in the notebook
%matplotlib inline

# Load the trained model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp3/weights/best.pt')

# Define the path to the video
video_path = '/kaggle/input/videotraffic/Traffic.mp4'  # Replace with your video path

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

frame_count = 0
total_vehicles = 0  # Initialize total vehicle counter

# Variables for FPS calculation
start_time = time.time()
fps = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Start timing for this frame processing
    frame_start_time = time.time()

    # Perform inference
    results = model(frame)

    # Get the number of vehicles detected in the current frame
    vehicle_count = len(results.xyxy[0])
    total_vehicles += vehicle_count  # Update the total vehicle count

    # Calculate FPS
    frame_time = time.time() - frame_start_time
    fps = 1 / frame_time

    # Convert BGR to RGB for matplotlib display
    img = cv2.cvtColor(np.squeeze(results.render()), cv2.COLOR_BGR2RGB)

    # Add FPS to the image
    cv2.putText(img, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the frame with YOLOv5 detections
    plt.figure(figsize=(10, 6))
    plt.imshow(img)
    plt.title(f"Frame {frame_count}: {vehicle_count} vehicles detected | FPS: {fps:.2f}")
    plt.axis('off')
    plt.show()  # Ensure the plot is shown

    # Clear the plot to avoid overlap in subsequent iterations
    plt.clf()

    frame_count += 1
    if frame_count >= 10:  # Display first 10 frames for example
        break

cap.release()

# Calculate average FPS
end_time = time.time()
avg_fps = frame_count / (end_time - start_time)
print(f"Total vehicles detected: {total_vehicles}")
print(f"Average FPS: {avg_fps:.2f}")

# **Example Code to Simulate Traffic Signal Management:**

In [ ]:
import torch
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

# Simulate traffic signals for 4 roads
traffic_signals = {
    'road_1': {'vehicles': 0, 'signal': 'RED'},
    'road_2': {'vehicles': 0, 'signal': 'RED'},
    'road_3': {'vehicles': 0, 'signal': 'RED'},
    'road_4': {'vehicles': 0, 'signal': 'RED'}
}

# Function to update traffic lights based on vehicle count
def update_traffic_signals():
    print("\nUpdating Traffic Signals...")
    # Sort roads by vehicle count (most to least vehicles)
    sorted_roads = sorted(traffic_signals, key=lambda x: traffic_signals[x]['vehicles'], reverse=True)

    # Set the first road with the most vehicles to green, others to red
    for road in traffic_signals:
        if road == sorted_roads[0]:
            traffic_signals[road]['signal'] = 'GREEN'
        else:
            traffic_signals[road]['signal'] = 'RED'

    print("Traffic Signal Update:")
    for road, info in traffic_signals.items():
        print(f"  {road}: Signal = {info['signal']}, Vehicles = {info['vehicles']}")

# Load the trained model
print("Loading YOLOv5 model...")
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp3/weights/best.pt')
print("Model loaded successfully!")

# Simulate each road with its own video feed
video_paths = {
    'road_1': '/kaggle/input/videotraffic/Traffic.mp4',
    'road_2': '/kaggle/input/videotraffic/Traffic2.mp4',
    'road_3': '/kaggle/input/videotraffic/Traffic3.mp4',
    'road_4': '/kaggle/input/videotraffic/Traffic4.mp4'
}

# Initialize video captures
caps = {road: cv2.VideoCapture(path) for road, path in video_paths.items()}

# Function to skip frames based on the video FPS and the required skip time (3 seconds)
def skip_frames(caps, skip_time):
    for road, cap in caps.items():
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        skip_frames = int(fps * skip_time)
        for _ in range(skip_frames):
            cap.grab()

# Process video frames
skip_time = 3  # Skip 3 seconds of video
iterations = 10  # Number of iterations

for iteration in range(iterations):
    print(f"\nIteration {iteration + 1}")
    for road, cap in caps.items():
        # Read a frame from the video feed
        ret, frame = cap.read()
        if not ret:
            print(f"  End of video for {road}")
            continue

        # Perform inference to detect vehicles
        print(f"  Detecting vehicles on {road}...")
        results = model(frame)
        vehicle_count = len(results.xyxy[0])

        # Update vehicle count for the road
        traffic_signals[road]['vehicles'] = vehicle_count
        print(f"  {road}: {vehicle_count} vehicles detected")

        # Render the results on the frame
        rendered_frame = results.render()[0]  # Render the bounding boxes and labels

        # Convert BGR frame to RGB for Matplotlib
        rendered_frame_rgb = cv2.cvtColor(rendered_frame, cv2.COLOR_BGR2RGB)

        # Display the frame using Matplotlib
        plt.figure(figsize=(10, 6))
        plt.imshow(rendered_frame_rgb)
        plt.title(f'{road} Frame with Detection')
        plt.axis('off')
        plt.show()

    # Skip frames equivalent to 3 seconds for each video
    skip_frames(caps, skip_time)

    # Update traffic signals based on vehicle count
    update_traffic_signals()

    # Introduce a delay of 1 second between each iteration
    time.sleep(1)  # Delay of 1 second before processing the next frame

# Release video captures
for cap in caps.values():
    cap.release()

print("End of program")

### Milestone 2

In [ ]:
# Install the necessary tracking package
!pip install deep-sort-realtime

# Add the Multi-Object Tracking cell
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict
import math

# Ensure matplotlib works in the notebook
%matplotlib inline

# Initialize DeepSORT tracker
tracker = DeepSort(
    max_age=30,              # Maximum number of frames to keep track when no detection
    n_init=3,                # Number of consecutive detections before track is confirmed
    nn_budget=100,           # Maximum size of appearance descriptors gallery
    embedder="mobilenet",    # Feature extractor model
    nms_max_overlap=1.0,     # Non-max suppression threshold (1.0 = no NMS)
    max_cosine_distance=0.2  # Threshold for appearance feature comparison
)

# Dictionary to store trajectories for each track ID
trajectories = defaultdict(list)

# Dictionary to store speed for each track ID
speeds = defaultdict(float)

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

# Function to estimate vehicle speed based on pixel displacement
def estimate_speed(traj_points, fps):
    if len(traj_points) < 5:
        return 0.0

    # Calculate displacement over last 5 points
    distances = []
    for i in range(1, min(6, len(traj_points))):
        distances.append(calculate_distance(traj_points[-i], traj_points[-i-1]))

    # Average displacement per frame
    avg_displacement = sum(distances) / len(distances)

    # Convert to speed (pixels per second)
    speed = avg_displacement * fps

    return speed

# Load the trained model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp3/weights/best.pt')

# Define the path to the video
video_path = '/kaggle/input/videotraffic/Traffic.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Video FPS: {fps}, Resolution: {frame_width}x{frame_height}")

# Variables for tracking statistics
frame_count = 0
total_vehicles = 0
total_unique_vehicles = 0
processing_times = []

# For visualization
colors = np.random.randint(0, 255, size=(100, 3), dtype=np.uint8)

# Create a video writer (optional, if you want to save the output)
# out = cv2.VideoWriter('tracked_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Process video frames
while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Start timing for this frame processing
    frame_start_time = time.time()

    # Perform detection inference
    results = model(frame)

    # Get detections as numpy array [x1, y1, x2, y2, confidence, class]
    detections = results.xyxy[0].cpu().numpy()

    # Format detections for DeepSORT [x1, y1, width, height, confidence, class]
    detection_list = []
    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection
        detection_list.append(
            ([x1, y1, x2 - x1, y2 - y1], conf, int(cls))
        )

    # Update tracker with detections
    tracks = tracker.update_tracks(detection_list, frame=frame)

    # Count active tracks in this frame
    active_tracks = [track for track in tracks if track.is_confirmed()]
    frame_vehicle_count = len(active_tracks)
    total_vehicles += frame_vehicle_count

    # Keep track of unique vehicles
    for track in active_tracks:
        track_id = track.track_id
        if track_id not in trajectories:
            total_unique_vehicles += 1

        # Get track position (center of bounding box)
        bbox = track.to_ltrb()
        x1, y1, x2, y2 = bbox
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)

        # Add point to trajectory
        trajectories[track_id].append((center_x, center_y))

        # Update speed estimate
        speeds[track_id] = estimate_speed(trajectories[track_id], fps)

    # Draw tracking results on the frame
    tracked_frame = frame.copy()

    for track in active_tracks:
        track_id = track.track_id
        bbox = track.to_ltrb()
        x1, y1, x2, y2 = bbox

        # Get color for this track ID
        color = colors[track_id % len(colors)].tolist()

        # Draw bounding box
        cv2.rectangle(tracked_frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

        # Add ID and speed text
        speed_text = f"ID:{track_id} Speed:{speeds[track_id]:.1f}"
        cv2.putText(tracked_frame, speed_text, (int(x1), int(y1)-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Draw trajectory
        if len(trajectories[track_id]) > 1:
            # Draw the trajectory line
            for i in range(1, len(trajectories[track_id])):
                pt1 = trajectories[track_id][i-1]
                pt2 = trajectories[track_id][i]
                cv2.line(tracked_frame, pt1, pt2, color, 2)

    # Calculate FPS
    processing_time = time.time() - frame_start_time
    processing_times.append(processing_time)
    fps_current = 1.0 / processing_time if processing_time > 0 else 0

    # Add FPS text to the frame
    cv2.putText(tracked_frame, f"FPS: {fps_current:.2f}", (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Add detection and tracking info
    cv2.putText(tracked_frame, f"Tracking: {frame_vehicle_count} vehicles", (10, 60),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Convert BGR to RGB for matplotlib display
    tracked_frame_rgb = cv2.cvtColor(tracked_frame, cv2.COLOR_BGR2RGB)

    # Display the frame
    plt.figure(figsize=(12, 8))
    plt.imshow(tracked_frame_rgb)
    plt.title(f"Frame {frame_count}: {frame_vehicle_count} vehicles tracked | FPS: {fps_current:.2f}")
    plt.axis('off')
    plt.show()
    plt.clf()

    # Optional: Save frame to output video
    # out.write(tracked_frame)

    frame_count += 1

    # For demo purposes, process a limited number of frames
    if frame_count >= 20:  # Increase this to process more frames
        break

# Release video and cleanup
cap.release()
# out.release()  # Uncomment if saving video

# Calculate and display performance metrics
avg_processing_time = sum(processing_times) / len(processing_times)
avg_fps = 1.0 / avg_processing_time if avg_processing_time > 0 else 0

print(f"Processed {frame_count} frames")
print(f"Average processing time: {avg_processing_time:.3f} seconds per frame")
print(f"Average FPS: {avg_fps:.2f}")
print(f"Total vehicles tracked: {total_unique_vehicles}")
print(f"Average vehicles per frame: {total_vehicles / frame_count:.2f}")

# Generate trajectory visualization
plt.figure(figsize=(12, 8))
plt.title("Vehicle Trajectories")

# Create a blended background (optional, average of first and last frame)
background = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
# Background could be created by blending first/last frames if needed

# Draw all trajectories on a single image
for track_id, traj in trajectories.items():
    if len(traj) < 3:  # Skip very short trajectories
        continue

    # Get color for this track
    color = colors[track_id % len(colors)].tolist()

    # Convert to numpy for easier indexing
    traj_np = np.array(traj)

    # Plot trajectory points
    plt.plot(traj_np[:, 0], traj_np[:, 1], '-', color=[c/255.0 for c in color], linewidth=2,
             label=f"Vehicle {track_id}")

    # Mark start and end points
    plt.plot(traj_np[0, 0], traj_np[0, 1], 'o', color=[c/255.0 for c in color], markersize=8)
    plt.plot(traj_np[-1, 0], traj_np[-1, 1], 's', color=[c/255.0 for c in color], markersize=8)

# Set plot properties
plt.xlim(0, frame_width)
plt.ylim(frame_height, 0)  # Invert y-axis to match image coordinates
plt.grid(True)
plt.tight_layout()
plt.show()

# Advanced: Analyze trajectories for patterns
# Calculate average direction and speed for each trajectory
trajectory_stats = {}

for track_id, traj in trajectories.items():
    if len(traj) < 5:  # Need enough points for meaningful analysis
        continue

    # Calculate overall displacement vector
    start_point = np.array(traj[0])
    end_point = np.array(traj[-1])
    displacement = end_point - start_point

    # Calculate direction angle in degrees (0 is right, 90 is up)
    direction = math.degrees(math.atan2(-displacement[1], displacement[0]))  # Negative Y for image coordinates

    # Calculate average speed
    total_distance = 0
    for i in range(1, len(traj)):
        total_distance += calculate_distance(traj[i-1], traj[i])

    avg_speed = total_distance / len(traj)

    # Store stats
    trajectory_stats[track_id] = {
        'direction': direction,
        'avg_speed': avg_speed,
        'displacement': np.linalg.norm(displacement),
        'trajectory_length': len(traj)
    }

# Print trajectory statistics
print("\nTrajectory Statistics:")
for track_id, stats in trajectory_stats.items():
    print(f"Vehicle {track_id}:")
    print(f"  Direction: {stats['direction']:.1f} degrees")
    print(f"  Average Speed: {stats['avg_speed']:.1f} pixels/frame")
    print(f"  Displacement: {stats['displacement']:.1f} pixels")
    print(f"  Tracked for {stats['trajectory_length']} frames")

### Milestone 3

In [ ]:
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict
import math

# Ensure matplotlib works in the notebook
%matplotlib inline

# Step 1: Define behavior analysis functions
def analyze_trajectory_direction(trajectory, window_size=10):
    """Analyze trajectory to determine if vehicle is turning and in which direction"""
    if len(trajectory) < window_size:
        return "Unknown"

    # Get recent trajectory points
    recent_traj = trajectory[-window_size:]

    # Calculate direction changes
    angles = []
    for i in range(1, len(recent_traj)):
        dx = recent_traj[i][0] - recent_traj[i-1][0]
        dy = recent_traj[i][1] - recent_traj[i-1][1]
        angle = math.atan2(dy, dx)
        angles.append(angle)

    # Calculate angle difference between start and end
    angle_diff = abs(angles[-1] - angles[0])

    # Classify turn direction
    if angle_diff > 0.3:  # Threshold for turn detection (in radians)
        # Determine turn direction
        if (angles[-1] - angles[0]) > 0:
            return "Turning Left"
        else:
            return "Turning Right"
    else:
        return "Going Straight"

def detect_red_light_violation(trajectory, current_pos, stop_line_y, light_state):
    """Detect if a vehicle might run a red light"""
    if light_state != "RED" or len(trajectory) < 5:
        return False

    # Check if vehicle is approaching the stop line
    if current_pos[1] < stop_line_y and current_pos[1] > stop_line_y - 100:
        # Calculate speed
        recent_traj = trajectory[-5:]
        distances = []
        for i in range(1, len(recent_traj)):
            distances.append(calculate_distance(recent_traj[i], recent_traj[i-1]))

        avg_speed = sum(distances) / len(distances)

        # If approaching fast and not slowing down
        if avg_speed > 5.0:  # Threshold for "fast" approach
            return True

    return False

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

# Function to estimate vehicle speed based on pixel displacement
def estimate_speed(traj_points, fps):
    if len(traj_points) < 5:
        return 0.0

    # Calculate displacement over last 5 points
    distances = []
    for i in range(1, min(6, len(traj_points))):
        distances.append(calculate_distance(traj_points[-i], traj_points[-i-1]))

    # Average displacement per frame
    avg_displacement = sum(distances) / len(distances)

    # Convert to speed (pixels per second)
    speed = avg_displacement * fps

    return speed

# Step 2: Define intersection zones
def define_intersection_zones(frame_width, frame_height):
    """Define virtual zones for traffic analysis based on frame dimensions"""

    # Example for a four-way intersection
    intersection = {
        'center': (frame_width//2, frame_height//2),
        'radius': 150,
        'stop_lines': {
            'north': frame_height//2 - 100,
            'south': frame_height//2 + 100,
            'east': frame_width//2 + 100,
            'west': frame_width//2 - 100
        },
        'light_states': {
            'north': 'RED',
            'south': 'RED',
            'east': 'GREEN',
            'west': 'RED'
        }
    }

    return intersection

# Function to visualize intersection zones
def draw_intersection_zones(frame, intersection):
    """Draw intersection zones on the frame"""
    frame_width = frame.shape[1]
    frame_height = frame.shape[0]

    # Draw center circle
    cv2.circle(frame, intersection['center'], 10, (0, 0, 255), -1)
    cv2.circle(frame, intersection['center'], intersection['radius'], (0, 0, 255), 2)

    # Draw stop lines
    cv2.line(frame,
             (0, intersection['stop_lines']['north']),
             (frame_width, intersection['stop_lines']['north']),
             (255, 0, 0), 2)
    cv2.line(frame,
             (0, intersection['stop_lines']['south']),
             (frame_width, intersection['stop_lines']['south']),
             (255, 0, 0), 2)
    cv2.line(frame,
             (intersection['stop_lines']['west'], 0),
             (intersection['stop_lines']['west'], frame_height),
             (255, 0, 0), 2)
    cv2.line(frame,
             (intersection['stop_lines']['east'], 0),
             (intersection['stop_lines']['east'], frame_height),
             (255, 0, 0), 2)

    # Add traffic light indicators
    light_positions = {
        'north': (frame_width//2 - 50, intersection['stop_lines']['north'] - 20),
        'south': (frame_width//2 + 50, intersection['stop_lines']['south'] + 20),
        'east': (intersection['stop_lines']['east'] + 20, frame_height//2 - 50),
        'west': (intersection['stop_lines']['west'] - 20, frame_height//2 + 50)
    }

    for direction, position in light_positions.items():
        # Determine light color
        if intersection['light_states'][direction] == 'RED':
            light_color = (0, 0, 255)  # Red
        elif intersection['light_states'][direction] == 'YELLOW':
            light_color = (0, 255, 255)  # Yellow
        else:  # GREEN
            light_color = (0, 255, 0)  # Green

        # Draw light indicator
        cv2.circle(frame, position, 15, light_color, -1)
        cv2.circle(frame, position, 15, (255, 255, 255), 2)

    return frame

# Function to update traffic light states
def update_traffic_lights(intersection, frame_count, cycle_length=150):
    """Update traffic light states based on a simple cycle"""
    # Simple traffic light cycle: N/S green, E/W red, then switch
    cycle_position = frame_count % cycle_length

    # First half of cycle: N/S green, E/W red
    if cycle_position < cycle_length // 2:
        intersection['light_states']['north'] = 'GREEN'
        intersection['light_states']['south'] = 'GREEN'
        intersection['light_states']['east'] = 'RED'
        intersection['light_states']['west'] = 'RED'

        # Add yellow transition
        if cycle_position > (cycle_length // 2) - 30:
            intersection['light_states']['north'] = 'YELLOW'
            intersection['light_states']['south'] = 'YELLOW'
    # Second half: N/S red, E/W green
    else:
        intersection['light_states']['north'] = 'RED'
        intersection['light_states']['south'] = 'RED'
        intersection['light_states']['east'] = 'GREEN'
        intersection['light_states']['west'] = 'GREEN'

        # Add yellow transition
        if cycle_position > cycle_length - 30:
            intersection['light_states']['east'] = 'YELLOW'
            intersection['light_states']['west'] = 'YELLOW'

    return intersection

# Step 3-5: Main tracking and behavior prediction function
def track_and_predict_behavior(video_path, model, max_frames=50):
    """Main function to track vehicles and predict their behavior"""

    # Initialize DeepSORT tracker
    tracker = DeepSort(
        max_age=30,              # Maximum number of frames to keep track when no detection
        n_init=3,                # Number of consecutive detections before track is confirmed
        nn_budget=100,           # Maximum size of appearance descriptors gallery
        embedder="mobilenet",    # Feature extractor model
        nms_max_overlap=1.0,     # Non-max suppression threshold (1.0 = no NMS)
        max_cosine_distance=0.2  # Threshold for appearance feature comparison
    )

    # Dictionary to store trajectories for each track ID
    trajectories = defaultdict(list)

    # Dictionary to store speed for each track ID
    speeds = defaultdict(float)

    # Dictionary to store behavior for each track ID
    behaviors = defaultdict(str)

    # Dictionary to store violations for each track ID
    violations = defaultdict(bool)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Video FPS: {fps}, Resolution: {frame_width}x{frame_height}")

    # Define intersection zones
    intersection = define_intersection_zones(frame_width, frame_height)

    # Variables for tracking statistics
    frame_count = 0
    total_vehicles = 0
    total_unique_vehicles = 0
    total_violations = 0
    processing_times = []

    # For visualization
    colors = np.random.randint(0, 255, size=(100, 3), dtype=np.uint8)

    # Process video frames
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Start timing for this frame processing
        frame_start_time = time.time()

        # Update traffic light states
        intersection = update_traffic_lights(intersection, frame_count)

        # Perform detection inference
        results = model(frame)

        # Get detections as numpy array [x1, y1, x2, y2, confidence, class]
        detections = results.xyxy[0].cpu().numpy()

        # Format detections for DeepSORT [x1, y1, width, height, confidence, class]
        detection_list = []
        for detection in detections:
            x1, y1, x2, y2, conf, cls = detection
            detection_list.append(
                ([x1, y1, x2 - x1, y2 - y1], conf, int(cls))
            )

        # Update tracker with detections
        tracks = tracker.update_tracks(detection_list, frame=frame)

        # Count active tracks in this frame
        active_tracks = [track for track in tracks if track.is_confirmed()]
        frame_vehicle_count = len(active_tracks)
        total_vehicles += frame_vehicle_count

        # Draw tracking results on the frame
        tracked_frame = frame.copy()

        # Draw intersection zones
        tracked_frame = draw_intersection_zones(tracked_frame, intersection)

        # Keep track of unique vehicles and update trajectories
        for track in active_tracks:
            track_id = track.track_id
            bbox = track.to_ltrb()
            x1, y1, x2, y2 = bbox

            # Get track position (center of bounding box)
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            current_pos = (center_x, center_y)

            # Add point to trajectory
            trajectories[track_id].append(current_pos)

            # Update speed estimate
            speeds[track_id] = estimate_speed(trajectories[track_id], fps)

            # Count unique vehicles
            if len(trajectories[track_id]) == 1:
                total_unique_vehicles += 1

            # Get color for this track ID
            color = colors[track_id % len(colors)].tolist()

            # Determine which region of the intersection the vehicle is in
            if center_x < intersection['center'][0] and center_y < intersection['center'][1]:
                approach_direction = 'north'
            elif center_x >= intersection['center'][0] and center_y < intersection['center'][1]:
                approach_direction = 'east'
            elif center_x < intersection['center'][0] and center_y >= intersection['center'][1]:
                approach_direction = 'west'
            else:
                approach_direction = 'south'

            # Step 3: Analyze behavior if we have enough trajectory history
            if len(trajectories[track_id]) >= 10:
                behaviors[track_id] = analyze_trajectory_direction(trajectories[track_id])

                # Step 4: Check for red light violation
                stop_line_y = intersection['stop_lines'][approach_direction]
                light_state = intersection['light_states'][approach_direction]

                # Detect violation
                new_violation = detect_red_light_violation(
                    trajectories[track_id],
                    current_pos,
                    stop_line_y,
                    light_state
                )

                # Count new violations
                if new_violation and not violations[track_id]:
                    total_violations += 1

                violations[track_id] = new_violation

            # Step 5: Integrate behavior prediction into tracking visualization
            # Draw bounding box (red for violation, normal color otherwise)
            box_color = (0, 0, 255) if violations[track_id] else color
            cv2.rectangle(tracked_frame, (int(x1), int(y1)), (int(x2), int(y2)), box_color, 2)

            # Add ID and behavior text
            behavior_text = f"ID:{track_id} {behaviors[track_id]}"
            cv2.putText(tracked_frame, behavior_text, (int(x1), int(y1)-10),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Add speed text
            speed_text = f"Speed:{speeds[track_id]:.1f}"
            cv2.putText(tracked_frame, speed_text, (int(x1), int(y1)-30),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Add violation warning
            if violations[track_id]:
                cv2.putText(tracked_frame, "VIOLATION!", (int(x1), int(y1)-50),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Draw trajectory
            if len(trajectories[track_id]) > 1:
                # Draw the trajectory line
                for i in range(1, len(trajectories[track_id])):
                    pt1 = trajectories[track_id][i-1]
                    pt2 = trajectories[track_id][i]
                    cv2.line(tracked_frame, pt1, pt2, color, 2)

        # Calculate processing time and FPS
        processing_time = time.time() - frame_start_time
        processing_times.append(processing_time)
        fps_current = 1.0 / processing_time if processing_time > 0 else 0

        # Add FPS text to the frame
        cv2.putText(tracked_frame, f"FPS: {fps_current:.2f}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Add detection and tracking info
        cv2.putText(tracked_frame, f"Tracking: {frame_vehicle_count} vehicles", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Add violation counter
        cv2.putText(tracked_frame, f"Violations: {total_violations}", (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Convert BGR to RGB for matplotlib display
        tracked_frame_rgb = cv2.cvtColor(tracked_frame, cv2.COLOR_BGR2RGB)

        # Display the frame
        plt.figure(figsize=(12, 8))
        plt.imshow(tracked_frame_rgb)
        plt.title(f"Frame {frame_count}: {frame_vehicle_count} vehicles tracked | FPS: {fps_current:.2f}")
        plt.axis('off')
        plt.show()
        plt.clf()

        frame_count += 1

        # For demo purposes, process a limited number of frames
        if frame_count >= max_frames:  # Increase this to process more frames
            break

    # Release video
    cap.release()

    # Calculate and display performance metrics
    avg_processing_time = sum(processing_times) / len(processing_times) if processing_times else 0
    avg_fps = 1.0 / avg_processing_time if avg_processing_time > 0 else 0

    print(f"Processed {frame_count} frames")
    print(f"Average processing time: {avg_processing_time:.3f} seconds per frame")
    print(f"Average FPS: {avg_fps:.2f}")
    print(f"Total unique vehicles tracked: {total_unique_vehicles}")
    print(f"Average vehicles per frame: {total_vehicles / frame_count:.2f}")
    print(f"Total red light violations detected: {total_violations}")

    # Step 6: Calculate behavior prediction metrics
    print("\nBehavior Prediction Statistics:")
    behavior_counts = defaultdict(int)
    for track_id, behavior in behaviors.items():
        behavior_counts[behavior] += 1

    for behavior, count in behavior_counts.items():
        print(f"  {behavior}: {count} vehicles")

    print(f"\nViolation rate: {total_violations / total_unique_vehicles * 100:.1f}% of vehicles")

    # Return data for further analysis
    return {
        'trajectories': trajectories,
        'behaviors': behaviors,
        'violations': violations,
        'speeds': speeds,
        'metrics': {
            'total_frames': frame_count,
            'total_unique_vehicles': total_unique_vehicles,
            'total_violations': total_violations,
            'avg_fps': avg_fps
        }
    }

# Function to visualize all trajectories with behavior information
def visualize_trajectories(trajectory_data, frame_width, frame_height):
    """Create a summary visualization of all vehicle trajectories and their behaviors"""

    trajectories = trajectory_data['trajectories']
    behaviors = trajectory_data['behaviors']
    violations = trajectory_data['violations']

    # Set up the plot
    plt.figure(figsize=(12, 8))
    plt.title("Vehicle Trajectories and Behaviors")

    # Define colors for different behaviors
    behavior_colors = {
        "Going Straight": "blue",
        "Turning Left": "green",
        "Turning Right": "purple",
        "Unknown": "gray"
    }

    # Draw intersection
    intersection = define_intersection_zones(frame_width, frame_height)
    plt.plot(intersection['center'][0], intersection['center'][1], 'ro', markersize=10)
    plt.plot([0, frame_width], [intersection['stop_lines']['north'], intersection['stop_lines']['north']], 'r--')
    plt.plot([0, frame_width], [intersection['stop_lines']['south'], intersection['stop_lines']['south']], 'r--')
    plt.plot([intersection['stop_lines']['west'], intersection['stop_lines']['west']], [0, frame_height], 'r--')
    plt.plot([intersection['stop_lines']['east'], intersection['stop_lines']['east']], [0, frame_height], 'r--')

    # Draw all trajectories on a single image
    for track_id, traj in trajectories.items():
        if len(traj) < 3:  # Skip very short trajectories
            continue

        # Get behavior for this track
        behavior = behaviors.get(track_id, "Unknown")

        # Get color based on behavior
        color = behavior_colors.get(behavior, "gray")

        # Mark violations with a red outline
        is_violation = violations.get(track_id, False)

        # Convert to numpy for easier indexing
        traj_np = np.array(traj)

        # Plot trajectory
        plt.plot(traj_np[:, 0], traj_np[:, 1], '-', color=color, linewidth=2,
                 label=f"Vehicle {track_id}: {behavior}" if track_id % 5 == 0 else "")

        # Mark start and end points
        plt.plot(traj_np[0, 0], traj_np[0, 1], 'o', color=color, markersize=8)

        # Use a red square for the end point of vehicles with violations
        if is_violation:
            plt.plot(traj_np[-1, 0], traj_np[-1, 1], 's', color='red', markersize=10)
        else:
            plt.plot(traj_np[-1, 0], traj_np[-1, 1], 's', color=color, markersize=8)

    # Create a custom legend for behaviors
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='blue', lw=2, label='Going Straight'),
        Line2D([0], [0], color='green', lw=2, label='Turning Left'),
        Line2D([0], [0], color='purple', lw=2, label='Turning Right'),
        Line2D([0], [0], color='gray', lw=2, label='Unknown'),
        Line2D([0], [0], marker='s', color='red', label='Violation', markersize=8, linestyle='None')
    ]

    # Add legend with behavior types
    plt.legend(handles=legend_elements, loc='upper right')

    # Set plot properties
    plt.xlim(0, frame_width)
    plt.ylim(frame_height, 0)  # Invert y-axis to match image coordinates
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Example usage:
# Load the model and run tracking with behavior prediction
# model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp3/weights/best.pt')
# video_path = '/kaggle/input/videotraffic/Traffic.mp4'
# results = track_and_predict_behavior(video_path, model, max_frames=30)
# visualize_trajectories(results, 1280, 720)  # Use actual video dimensions

In [ ]:
# Load your trained model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp3/weights/best.pt')

# Define your video path
video_path = '/kaggle/input/videotraffic/Traffic.mp4'

# Run tracking with behavior prediction
results = track_and_predict_behavior(video_path, model, max_frames=30)

# Get video dimensions for visualization
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

# Visualize all trajectories with behavior information
visualize_trajectories(results, width, height)

###Milestone 4

In [ ]:
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict
import math
from scipy.signal import savgol_filter
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

# Ensure matplotlib works in the notebook
%matplotlib inline

# Step 7: Optimize tracker for occlusion handling
def create_optimized_tracker():
    """Create a DeepSORT tracker with optimized parameters for occlusion handling"""
    tracker = DeepSort(
        max_age=45,               # Increased from 30 to keep tracks alive longer during occlusions
        n_init=2,                 # Reduced from 3 to confirm tracks faster in difficult scenes
        nn_budget=150,            # Increased from 100 to store more appearance features
        embedder="mobilenet",     # Feature extractor model
        nms_max_overlap=0.8,      # Reduced from 1.0 to apply some non-max suppression
        max_cosine_distance=0.3,  # Increased from 0.2 to allow more appearance variation
        max_iou_distance=0.7,     # Explicitly set max IOU distance for better occlusion handling
        embedder_gpu=True,        # Use GPU for embedding computation if available
        embedder_model_name=None, # Use default model
        polygon=False             # Use rectangle (default) to represent objects
    )
    return tracker

# Step 9: Implement Kalman Filter for position prediction during occlusion
class VehicleKalmanFilter:
    """Kalman Filter implementation for vehicle position prediction during occlusion"""
    def __init__(self, initial_pos, dt=1.0):
        # 6 state variables [x, y, vx, vy, ax, ay], 2 measured variables [x, y]
        self.kf = KalmanFilter(dim_x=6, dim_z=2)

        # State transition matrix (position + velocity + acceleration model)
        self.kf.F = np.array([
            [1, 0, dt, 0, 0.5*dt**2, 0],
            [0, 1, 0, dt, 0, 0.5*dt**2],
            [0, 0, 1, 0, dt, 0],
            [0, 0, 0, 1, 0, dt],
            [0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 1]
        ])

        # Measurement function (we only measure position, not velocity or acceleration)
        self.kf.H = np.array([
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0]
        ])

        # Measurement noise
        self.kf.R = np.array([
            [10, 0],
            [0, 10]
        ])

        # Process noise
        self.kf.Q = np.eye(6) * 0.1

        # Process noise for position, velocity, and acceleration
        self.kf.Q[0:2, 0:2] *= 0.01  # position noise
        self.kf.Q[2:4, 2:4] *= 0.1   # velocity noise
        self.kf.Q[4:6, 4:6] *= 1.0   # acceleration noise

        # Initial state
        self.kf.x = np.array([initial_pos[0], initial_pos[1], 0, 0, 0, 0]).reshape(6, 1)

        # Initial state covariance
        self.kf.P *= 100

        # Store prediction history
        self.predictions = [initial_pos]
        self.measurements = [initial_pos]

        # Time step
        self.dt = dt

    def update(self, measurement):
        """Update the filter with a new measurement"""
        self.kf.predict()
        self.kf.update(np.array([measurement[0], measurement[1]]))

        # Store measurement and current state
        self.measurements.append(measurement)
        self.predictions.append((self.kf.x[0, 0], self.kf.x[1, 0]))

        return (self.kf.x[0, 0], self.kf.x[1, 0])

    def predict(self, steps=1):
        """Predict position several steps into the future"""
        # Save current state
        current_x = self.kf.x.copy()
        current_P = self.kf.P.copy()

        # Make predictions
        for _ in range(steps):
            self.kf.predict()

        # Get prediction
        prediction = (self.kf.x[0, 0], self.kf.x[1, 0])

        # Restore state
        self.kf.x = current_x
        self.kf.P = current_P

        return prediction

# Function to smooth trajectory using Savitzky-Golay filter
def smooth_trajectory(traj, window_size=15, poly_order=3):
    """Smooth a trajectory using Savitzky-Golay filter"""
    if len(traj) < window_size:
        return traj

    # Convert to numpy array
    traj_np = np.array(traj)

    # Smooth x and y separately
    x_smooth = savgol_filter(traj_np[:, 0], window_size, poly_order)
    y_smooth = savgol_filter(traj_np[:, 1], window_size, poly_order)

    # Combine back to pairs
    smoothed_traj = list(zip(x_smooth, y_smooth))

    return smoothed_traj

# Step 8-10: Enhanced tracking with occlusion handling and prediction
def track_with_occlusion_handling(video_path, model, max_frames=50, output_path=None):
    """Advanced tracking function with occlusion handling and prediction"""

    # Create optimized tracker
    tracker = create_optimized_tracker()

    # Dictionary to store trajectories for each track ID
    trajectories = defaultdict(list)

    # Dictionary to store smoothed trajectories
    smooth_trajectories = defaultdict(list)

    # Dictionary to store Kalman filters for each track
    kalman_filters = {}

    # Dictionary to store visibility status (True = visible, False = occluded)
    is_visible = defaultdict(bool)

    # Dictionary to store occlusion count and duration
    occlusion_stats = defaultdict(lambda: {"count": 0, "total_frames": 0, "current_streak": 0})

    # Dictionary to store speed for each track ID
    speeds = defaultdict(float)

    # Dictionary to store behavior for each track ID
    behaviors = defaultdict(str)

    # Dictionary to store violations for each track ID
    violations = defaultdict(bool)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Video FPS: {fps}, Resolution: {frame_width}x{frame_height}")

    # Define intersection zones
    intersection = define_intersection_zones(frame_width, frame_height)

    # Set up video writer if output path is provided
    video_writer = None
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Variables for tracking statistics
    frame_count = 0
    total_vehicles = 0
    total_unique_vehicles = 0
    total_violations = 0
    total_occlusions = 0
    processing_times = []

    # For visualization
    colors = np.random.randint(0, 255, size=(100, 3), dtype=np.uint8)

    # Process video frames
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Start timing for this frame processing
        frame_start_time = time.time()

        # Update traffic light states
        intersection = update_traffic_lights(intersection, frame_count)

        # Perform detection inference
        results = model(frame)

        # Get detections as numpy array [x1, y1, x2, y2, confidence, class]
        detections = results.xyxy[0].cpu().numpy()

        # Format detections for DeepSORT [x1, y1, width, height, confidence, class]
        detection_list = []
        for detection in detections:
            x1, y1, x2, y2, conf, cls = detection
            detection_list.append(
                ([x1, y1, x2 - x1, y2 - y1], conf, int(cls))
            )

        # Update tracker with detections
        tracks = tracker.update_tracks(detection_list, frame=frame)

        # Count active tracks in this frame
        active_tracks = [track for track in tracks if track.is_confirmed()]
        frame_vehicle_count = len(active_tracks)
        total_vehicles += frame_vehicle_count

        # Draw tracking results on the frame
        tracked_frame = frame.copy()

        # Draw intersection zones
        tracked_frame = draw_intersection_zones(tracked_frame, intersection)

        # Update visibility status for all tracks
        current_track_ids = set()

        # Process each active track
        for track in active_tracks:
            track_id = track.track_id
            current_track_ids.add(track_id)

            # Mark track as visible
            is_visible[track_id] = True

            # Reset occlusion streak counter
            occlusion_stats[track_id]["current_streak"] = 0

            bbox = track.to_ltrb()
            x1, y1, x2, y2 = bbox

            # Get track position (center of bounding box)
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            current_pos = (center_x, center_y)

            # Create Kalman filter for new tracks
            if track_id not in kalman_filters:
                kalman_filters[track_id] = VehicleKalmanFilter(current_pos, dt=1.0/fps)

            # Update Kalman filter with new position
            kalman_pos = kalman_filters[track_id].update(current_pos)

            # Add point to trajectory
            trajectories[track_id].append(current_pos)

            # Update smoothed trajectory
            if len(trajectories[track_id]) > 5:
                smooth_trajectories[track_id] = smooth_trajectory(trajectories[track_id])
            else:
                smooth_trajectories[track_id] = trajectories[track_id].copy()

            # Update speed estimate
            speeds[track_id] = estimate_speed(trajectories[track_id], fps)

            # Count unique vehicles
            if len(trajectories[track_id]) == 1:
                total_unique_vehicles += 1

            # Continue with behavior prediction and visualization...

        # Step 9: Handle occlusions - check for tracks that are not visible in this frame but were active recently
        all_known_tracks = set(trajectories.keys())
        occluded_tracks = all_known_tracks - current_track_ids

        for track_id in occluded_tracks:
            # Skip if track has been missing for too long (more than max_age frames)
            if frame_count - len(trajectories[track_id]) > tracker.tracker.max_age:
                continue

            # Track is occluded in this frame
            if is_visible[track_id]:  # First frame of occlusion
                is_visible[track_id] = False
                occlusion_stats[track_id]["count"] += 1
                total_occlusions += 1

            # Update occlusion streak counter
            occlusion_stats[track_id]["current_streak"] += 1
            occlusion_stats[track_id]["total_frames"] += 1

            # Use Kalman filter to predict position during occlusion
            if track_id in kalman_filters and len(trajectories[track_id]) > 0:
                # Predict position
                predicted_pos = kalman_filters[track_id].predict(steps=1)

                # Add predicted position to trajectory (marked differently)
                trajectories[track_id].append(predicted_pos)

                # Update smoothed trajectory
                smooth_trajectories[track_id] = smooth_trajectory(trajectories[track_id])

                # Draw occluded bounding box and trajectory with dashed lines
                last_width = last_height = 50  # Default size
                if len(trajectories[track_id]) > 1:
                    # Estimate object size from last known position
                    last_point = trajectories[track_id][-2]
                    predicted_point = predicted_pos
                    # Draw predicted bounding box (dashed)
                    cv2.rectangle(tracked_frame,
                                  (int(predicted_point[0] - last_width/2), int(predicted_point[1] - last_height/2)),
                                  (int(predicted_point[0] + last_width/2), int(predicted_point[1] + last_height/2)),
                                  (255, 255, 0), 1, cv2.LINE_DASHED)

                    # Draw text
                    cv2.putText(tracked_frame, f"Occluded ID:{track_id}",
                                (int(predicted_point[0] - last_width/2), int(predicted_point[1] - last_height/2 - 10)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

        # Draw all active tracks (both visible and predicted)
        for track_id in current_track_ids:
            # Get color for this track ID
            color = colors[track_id % len(colors)].tolist()

            # Get current bounding box for visible tracks
            for track in active_tracks:
                if track.track_id == track_id:
                    bbox = track.to_ltrb()
                    x1, y1, x2, y2 = bbox

                    # Determine which region of the intersection the vehicle is in
                    center_x = int((x1 + x2) / 2)
                    center_y = int((y1 + y2) / 2)

                    if center_x < intersection['center'][0] and center_y < intersection['center'][1]:
                        approach_direction = 'north'
                    elif center_x >= intersection['center'][0] and center_y < intersection['center'][1]:
                        approach_direction = 'east'
                    elif center_x < intersection['center'][0] and center_y >= intersection['center'][1]:
                        approach_direction = 'west'
                    else:
                        approach_direction = 'south'

                    # Step 3: Analyze behavior if we have enough trajectory history
                    if len(trajectories[track_id]) >= 10:
                        behaviors[track_id] = analyze_trajectory_direction(trajectories[track_id])

                        # Step 4: Check for red light violation
                        stop_line_y = intersection['stop_lines'][approach_direction]
                        light_state = intersection['light_states'][approach_direction]

                        # Detect violation
                        new_violation = detect_red_light_violation(
                            trajectories[track_id],
                            (center_x, center_y),
                            stop_line_y,
                            light_state
                        )

                        # Count new violations
                        if new_violation and not violations[track_id]:
                            total_violations += 1

                        violations[track_id] = new_violation

                    # Draw bounding box (red for violation, normal color otherwise)
                    box_color = (0, 0, 255) if violations[track_id] else color
                    cv2.rectangle(tracked_frame, (int(x1), int(y1)), (int(x2), int(y2)), box_color, 2)

                    # Add ID and behavior text
                    behavior_text = f"ID:{track_id} {behaviors[track_id]}"
                    cv2.putText(tracked_frame, behavior_text, (int(x1), int(y1)-10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                    # Add speed text
                    speed_text = f"Speed:{speeds[track_id]:.1f}"
                    cv2.putText(tracked_frame, speed_text, (int(x1), int(y1)-30),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                    # Add violation warning
                    if violations[track_id]:
                        cv2.putText(tracked_frame, "VIOLATION!", (int(x1), int(y1)-50),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Draw trajectory
            if len(smooth_trajectories[track_id]) > 1:
                # Draw the smoothed trajectory line
                for i in range(1, len(smooth_trajectories[track_id])):
                    pt1 = tuple(map(int, smooth_trajectories[track_id][i-1]))
                    pt2 = tuple(map(int, smooth_trajectories[track_id][i]))

                    # Draw predicted portions with dashed line if occluded
                    if i >= len(trajectories[track_id]) - occlusion_stats[track_id]["current_streak"]:
                        # Dashed line for predicted positions
                        for j in range(0, 10, 2):  # Draw dashed line
                            sub_pt1 = (int(pt1[0] + (pt2[0] - pt1[0]) * j / 10),
                                      int(pt1[1] + (pt2[1] - pt1[1]) * j / 10))
                            sub_pt2 = (int(pt1[0] + (pt2[0] - pt1[0]) * (j + 1) / 10),
                                      int(pt1[1] + (pt2[1] - pt1[1]) * (j + 1) / 10))
                            cv2.line(tracked_frame, sub_pt1, sub_pt2, (255, 255, 0), 2)
                    else:
                        # Solid line for actual positions
                        cv2.line(tracked_frame, pt1, pt2, color, 2)

        # Step 10: Add handling for congested scenes (draw density map)
        if frame_vehicle_count > 5:  # Only show heatmap in congested scenes
            # Create a density heatmap of vehicle positions
            heatmap = np.zeros((frame_height, frame_width), dtype=np.uint8)

            for track_id in current_track_ids:
                if len(trajectories[track_id]) > 0:
                    # Get last position
                    pos = trajectories[track_id][-1]
                    x, y = int(pos[0]), int(pos[1])

                    # Draw a gaussian blob at vehicle position
                    cv2.circle(heatmap, (x, y), 30, 255, -1)

            # Blur the heatmap to create density effect
            heatmap = cv2.GaussianBlur(heatmap, (99, 99), 0)

            # Normalize and colorize heatmap
            heatmap_normalized = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX)
            heatmap_colored = cv2.applyColorMap(heatmap_normalized, cv2.COLORMAP_JET)

            # Blend with original frame
            alpha = 0.3  # Transparency factor
            density_overlay = cv2.addWeighted(tracked_frame, 1 - alpha, heatmap_colored, alpha, 0)

            # Add density info text
            cv2.putText(density_overlay, "Traffic Density Map", (10, 120),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            tracked_frame = density_overlay

        # Calculate processing time and FPS
        processing_time = time.time() - frame_start_time
        processing_times.append(processing_time)
        fps_current = 1.0 / processing_time if processing_time > 0 else 0

        # Add FPS text to the frame
        cv2.putText(tracked_frame, f"FPS: {fps_current:.2f}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Add detection and tracking info
        cv2.putText(tracked_frame, f"Tracking: {frame_vehicle_count} vehicles", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Add violation counter
        cv2.putText(tracked_frame, f"Violations: {total_violations}", (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Add occlusion counter
        cv2.putText(tracked_frame, f"Occlusions: {total_occlusions}", (10, 150),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        # Write frame to output video if requested
        if video_writer:
            video_writer.write(tracked_frame)

        # Convert BGR to RGB for matplotlib display
        tracked_frame_rgb = cv2.cvtColor(tracked_frame, cv2.COLOR_BGR2RGB)

        # Display the frame
        plt.figure(figsize=(12, 8))
        plt.imshow(tracked_frame_rgb)
        plt.title(f"Frame {frame_count}: {frame_vehicle_count} vehicles tracked | FPS: {fps_current:.2f}")
        plt.axis('off')
        plt.show()
        plt.close()

        frame_count += 1

        # For demo purposes, process a limited number of frames
        if frame_count >= max_frames:  # Increase this to process more frames
            break

    # Release video resources
    cap.release()
    if video_writer:
        video_writer.release()

    # Calculate and display performance metrics
    avg_processing_time = sum(processing_times) / len(processing_times) if processing_times else 0
    avg_fps = 1.0 / avg_processing_time if avg_processing_time > 0 else 0

    print(f"Processed {frame_count} frames")
    print(f"Average processing time: {avg_processing_time:.3f} seconds per frame")
    print(f"Average FPS: {avg_fps:.2f}")
    print(f"Total unique vehicles tracked: {total_unique_vehicles}")
    print(f"Average vehicles per frame: {total_vehicles / frame_count:.2f}")
    print(f"Total red light violations detected: {total_violations}")
    print(f"Total occlusion events detected: {total_occlusions}")

    # Calculate occlusion statistics
    if occlusion_stats:
        total_occlusion_frames = sum(stats["total_frames"] for stats in occlusion_stats.values())
        avg_occlusion_duration = total_occlusion_frames / total_occlusions if total_occlusions > 0 else 0
        print(f"Average occlusion duration: {avg_occlusion_duration:.2f} frames")

        # Track recovery statistics
        recovered_tracks = sum(1 for track_id, stats in occlusion_stats.items()
                              if stats["count"] > 0 and is_visible[track_id])
        recovery_rate = recovered_tracks / total_occlusions if total_occlusions > 0 else 0
        print(f"Track recovery rate: {recovery_rate * 100:.1f}% ({recovered_tracks}/{total_occlusions})")

    # Return data for further analysis
    return {
        'trajectories': trajectories,
        'smooth_trajectories': smooth_trajectories,
        'behaviors': behaviors,
        'violations': violations,
        'speeds': speeds,
        'occlusion_stats': occlusion_stats,
        'metrics': {
            'total_frames': frame_count,
            'total_unique_vehicles': total_unique_vehicles,
            'total_violations': total_violations,
            'total_occlusions': total_occlusions,
            'avg_fps': avg_fps
        }
    }

# Functions from Milestone 3 needed for the implementation

def analyze_trajectory_direction(trajectory, window_size=10):
    """Analyze trajectory to determine if vehicle is turning and in which direction"""
    if len(trajectory) < window_size:
        return "Unknown"

    # Get recent trajectory points
    recent_traj = trajectory[-window_size:]

    # Calculate direction changes
    angles = []
    for i in range(1, len(recent_traj)):
        dx = recent_traj[i][0] - recent_traj[i-1][0]
        dy = recent_traj[i][1] - recent_traj[i-1][1]
        angle = math.atan2(dy, dx)
        angles.append(angle)

    # Calculate angle difference between start and end
    angle_diff = abs(angles[-1] - angles[0])

    # Classify turn direction
    if angle_diff > 0.3:  # Threshold for turn detection (in radians)
        # Determine turn direction
        if (angles[-1] - angles[0]) > 0:
            return "Turning Left"
        else:
            return "Turning Right"
    else:
        return "Going Straight"

def detect_red_light_violation(trajectory, current_pos, stop_line_y, light_state):
    """Detect if a vehicle might run a red light"""
    if light_state != "RED" or len(trajectory) < 5:
        return False

    # Check if vehicle is approaching the stop line
    if current_pos[1] < stop_line_y and current_pos[1] > stop_line_y - 100:
        # Calculate speed
        recent_traj = trajectory[-5:]
        distances = []
        for i in range(1, len(recent_traj)):
            distances.append(calculate_distance(recent_traj[i], recent_traj[i-1]))

        avg_speed = sum(distances) / len(distances)

        # If approaching fast and not slowing down
        if avg_speed > 5.0:  # Threshold for "fast" approach
            return True

    return False

def define_intersection_zones(frame_width, frame_height):
    """Define virtual zones for traffic analysis based on frame dimensions"""
    # Example for a four-way intersection
    intersection = {
        'center': (frame_width//2, frame_height//2),
        'radius': 150,
        'stop_lines': {
            'north': frame_height//2 - 100,
            'south': frame_height//2 + 100,
            'east': frame_width//2 + 100,
            'west': frame_width//2 - 100
        },
        'light_states': {
            'north': 'RED',
            'south': 'RED',
            'east': 'GREEN',
            'west': 'RED'
        }
    }

    return intersection

def draw_intersection_zones(frame, intersection):
    """Draw intersection zones on the frame"""
    frame_width = frame.shape[1]
    frame_height = frame.shape[0]

    # Draw center circle
    cv2.circle(frame, intersection['center'], 10, (0, 0, 255), -1)
    cv2.circle(frame, intersection['center'], intersection['radius'], (0, 0, 255), 2)

    # Draw stop lines
    cv2.line(frame,
             (0, intersection['stop_lines']['north']),
             (frame_width, intersection['stop_lines']['north']),
             (255, 0, 0), 2)
    cv2.line(frame,
             (0, intersection['stop_lines']['south']),
             (frame_width, intersection['stop_lines']['south']),
             (255, 0, 0), 2)
    cv2.line(frame,
             (intersection['stop_lines']['west'], 0),
             (intersection['stop_lines']['west'], frame_height),
             (255, 0, 0), 2)
    cv2.line(frame,
             (intersection['stop_lines']['east'], 0),
             (intersection['stop_lines']['east'], frame_height),
             (255, 0, 0), 2)

    # Add traffic light indicators
    light_positions = {
        'north': (frame_width//2 - 50, intersection['stop_lines']['north'] - 20),
        'south': (frame_width//2 + 50, intersection['stop_lines']['south'] + 20),
        'east': (intersection['stop_lines']['east'] + 20, frame_height//2 - 50),
        'west': (intersection['stop_lines']['west'] - 20, frame_height//2 + 50)
    }

    for direction, position in light_positions.items():
        # Determine light color
        if intersection['light_states'][direction] == 'RED':
            light_color = (0, 0, 255)  # Red
        elif intersection['light_states'][direction] == 'YELLOW':
            light_color = (0, 255, 255)  # Yellow
        else:  # GREEN
            light_color = (0, 255, 0)  # Green

        # Draw light indicator
        cv2.circle(frame, position, 15, light_color, -1)
        cv2.circle(frame, position, 15, (255, 255, 255), 2)

    return frame

def update_traffic_lights(intersection, frame_count, cycle_length=150):
    """Update traffic light states based on a simple cycle"""
    # Simple traffic light cycle: N/S green, E/W red, then switch
    cycle_position = frame_count % cycle_length

    # First half of cycle: N/S green, E/W red
    if cycle_position < cycle_length // 2:
        intersection['light_states']['north'] = 'GREEN'
        intersection['light_states']['south'] = 'GREEN'
        intersection['light_states']['east'] = 'RED'
        intersection['light_states']['west'] = 'RED'

        # Add yellow transition
        if cycle_position > (cycle_length // 2) - 30:
            intersection['light_states']['north'] = 'YELLOW'
            intersection['light_states']['south'] = 'YELLOW'
    # Second half: N/S red, E/W green
    else:
        intersection['light_states']['north'] = 'RED'
        intersection['light_states']['south'] = 'RED'
        intersection['light_states']['east'] = 'GREEN'
        intersection['light_states']['west'] = 'GREEN'

        # Add yellow transition
        if cycle_position > cycle_length - 30:
            intersection['light_states']['east'] = 'YELLOW'
            intersection['light_states']['west'] = 'YELLOW'

    return intersection

# Function to visualize occlusion handling results
def visualize_occlusion_results(tracking_results, frame_width, frame_height):
    """Create visualizations to analyze occlusion handling effectiveness"""

    trajectories = tracking_results['trajectories']
    smooth_trajectories = tracking_results['smooth_trajectories']
    occlusion_stats = tracking_results['occlusion_stats']

    # 1. Track continuity visualization - showing original and smoothed trajectories
    plt.figure(figsize=(14, 10))
    plt.title("Track Continuity: Original vs Smoothed Trajectories")

    # Draw intersection for reference
    intersection = define_intersection_zones(frame_width, frame_height)
    plt.plot(intersection['center'][0], intersection['center'][1], 'ko', markersize=10)
    plt.plot([0, frame_width], [intersection['stop_lines']['north'], intersection['stop_lines']['north']], 'k--')
    plt.plot([0, frame_width], [intersection['stop_lines']['south'], intersection['stop_lines']['south']], 'k--')
    plt.plot([intersection['stop_lines']['west'], intersection['stop_lines']['west']], [0, frame_height], 'k--')
    plt.plot([intersection['stop_lines']['east'], intersection['stop_lines']['east']], [0, frame_height], 'k--')

    # Select tracks that had occlusions for visualization
    occluded_track_ids = [track_id for track_id, stats in occlusion_stats.items()
                         if stats["count"] > 0 and len(trajectories[track_id]) > 10]

    # Limit to 5 tracks to avoid clutter
    selected_tracks = occluded_track_ids[:5] if len(occluded_track_ids) > 5 else occluded_track_ids

    for track_id in selected_tracks:
        # Get original trajectory
        orig_traj = np.array(trajectories[track_id])

        # Get smoothed trajectory
        smooth_traj = np.array(smooth_trajectories[track_id] if track_id in smooth_trajectories
                              else trajectories[track_id])

        # Plot original trajectory as dots
        plt.plot(orig_traj[:, 0], orig_traj[:, 1], '.', markersize=4, label=f"Original ID:{track_id}")

        # Plot smoothed trajectory as line
        plt.plot(smooth_traj[:, 0], smooth_traj[:, 1], '-', linewidth=2, label=f"Smoothed ID:{track_id}")

    plt.xlim(0, frame_width)
    plt.ylim(frame_height, 0)  # Invert y-axis to match image coordinates
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # 2. Occlusion statistics visualization
    plt.figure(figsize=(12, 6))

    # Filter to tracks with occlusions
    occlusion_counts = {track_id: stats["count"] for track_id, stats in occlusion_stats.items()
                       if stats["count"] > 0}

    # Sort by occlusion count
    sorted_occlusions = sorted(occlusion_counts.items(), key=lambda x: x[1], reverse=True)

    # Get top 15 tracks with most occlusions
    top_occluded = sorted_occlusions[:15]

    # Create bar chart
    track_ids = [f"ID:{tid}" for tid, _ in top_occluded]
    occlusion_counts = [count for _, count in top_occluded]

    plt.bar(track_ids, occlusion_counts, color='orange')
    plt.title("Occlusion Counts by Vehicle Track")
    plt.xlabel("Track ID")
    plt.ylabel("Number of Occlusion Events")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 3. Occlusion duration histogram
    occlusion_durations = [stats["total_frames"] / stats["count"] if stats["count"] > 0 else 0
                          for stats in occlusion_stats.values()]
    occlusion_durations = [d for d in occlusion_durations if d > 0]

    plt.figure(figsize=(10, 6))
    plt.hist(occlusion_durations, bins=20, color='green', alpha=0.7)
    plt.title("Distribution of Occlusion Durations")
    plt.xlabel("Average Occlusion Duration (frames)")
    plt.ylabel("Number of Tracks")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Test the implementation
def test_occlusion_handling(model, video_path, max_frames=30):
    """Test and evaluate occlusion handling"""

    # Make sure we have necessary imports and functions from previous implementation
    print("Starting occlusion handling test...")

    # Run tracking with occlusion handling
    results = track_with_occlusion_handling(
        video_path=video_path,
        model=model,
        max_frames=max_frames,
        output_path="occlusion_handling_output.mp4"
    )

    # Get video dimensions for visualization
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()

    # Analyze and visualize results
    visualize_occlusion_results(results, width, height)

    return results

In [ ]:
# Occlusion Evaluation Tests
# This script tests our tracker's performance in handling occlusions and complex scenarios

import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os
import time

# Import our implementation from Milestone 4
# (This assumes all the necessary functions from the milestone4-implementation.py file)

# Function to create synthetic occlusion scenarios for controlled testing
def create_synthetic_occlusion_test(base_video_path, output_path, occlusion_frames=[30, 60, 90],
                                   occlusion_size=0.3):
    """
    Create a test video with synthetic occlusions (black boxes) to evaluate tracking robustness

    Args:
        base_video_path: Path to the original video
        output_path: Path to save the modified video
        occlusion_frames: List of frame numbers where to introduce occlusions
        occlusion_size: Size of occlusion as a fraction of the frame size
    """
    # Open the input video
    cap = cv2.VideoCapture(base_video_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define occlusion size
    occlusion_width = int(width * occlusion_size)
    occlusion_height = int(height * occlusion_size)

    # Define occlusion positions (center of frame for simplicity)
    occlusion_x = width // 2 - occlusion_width // 2
    occlusion_y = height // 2 - occlusion_height // 2

    # Create output video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_number = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Check if we should add occlusion to this frame
        if frame_number in occlusion_frames:
            # Create a black occlusion rectangle
            frame[occlusion_y:occlusion_y+occlusion_height,
                 occlusion_x:occlusion_x+occlusion_width] = 0

            # Add occlusion indicator text
            cv2.putText(frame, "SYNTHETIC OCCLUSION", (occlusion_x, occlusion_y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        # Write the frame
        out.write(frame)
        frame_number += 1

        # For demo purposes only process first 120 frames
        if frame_number >= 120:
            break

    # Release resources
    cap.release()
    out.release()

    print(f"Created synthetic occlusion test video at {output_path}")
    return output_path

# Compare tracking performance with and without occlusion handling
def compare_tracking_performance(model, video_path, max_frames=30):
    """
    Compare tracking performance with regular tracking vs. occlusion-optimized tracking

    Args:
        model: The detection model to use
        video_path: Path to the video for testing
        max_frames: Number of frames to process
    """
    print("Comparing tracking methods...")

    # Track using standard DeepSORT (from Milestone 3)
    # This function should be imported from your Milestone 3 implementation
    standard_results = track_and_predict_behavior(
        video_path,
        model,
        max_frames=max_frames
    )

    # Track using occlusion-optimized tracking (from Milestone 4)
    occlusion_results = track_with_occlusion_handling(
        video_path,
        model,
        max_frames=max_frames
    )

    # Compare results
    standard_tracks = len(standard_results['trajectories'])
    occlusion_tracks = len(occlusion_results['trajectories'])

    # Calculate average track length
    standard_avg_len = sum(len(traj) for traj in standard_results['trajectories'].values()) / standard_tracks
    occlusion_avg_len = sum(len(traj) for traj in occlusion_results['trajectories'].values()) / occlusion_tracks

    print("\nTracking Performance Comparison:")
    print(f"Standard Tracking: {standard_tracks} tracks, avg length {standard_avg_len:.2f} frames")
    print(f"Occlusion-optimized: {occlusion_tracks} tracks, avg length {occlusion_avg_len:.2f} frames")

    # Calculate improvement percentage
    len_improvement = (occlusion_avg_len - standard_avg_len) / standard_avg_len * 100
    print(f"Track length improvement: {len_improvement:.2f}%")

    # Visualize track length comparison
    plt.figure(figsize=(12, 6))

    # Get track lengths
    standard_lengths = [len(traj) for traj in standard_results['trajectories'].values()]
    occlusion_lengths = [len(traj) for traj in occlusion_results['trajectories'].values()]

    # Create histograms
    bins = range(0, max(max(standard_lengths), max(occlusion_lengths)) + 5, 5)
    plt.hist(standard_lengths, bins=bins, alpha=0.5, label='Standard Tracking')
    plt.hist(occlusion_lengths, bins=bins, alpha=0.5, label='Occlusion-optimized')

    plt.title('Track Length Distribution Comparison')
    plt.xlabel('Track Length (frames)')
    plt.ylabel('Number of Tracks')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    return {
        'standard': standard_results,
        'occlusion': occlusion_results
    }

# Function to measure track fragmentation (ID switches)
def analyze_track_fragmentation(results, ground_truth=None):
    """
    Analyze how much track fragmentation occurs (when one physical object gets multiple IDs)

    Args:
        results: Tracking results
        ground_truth: Optional ground truth data (if available)
    """
    trajectories = results['trajectories']

    # If no ground truth, we'll use spatial-temporal overlap analysis
    # to estimate potential track fragments

    # Find tracks that could be fragments of the same physical object
    potential_fragments = []

    # Get all track IDs
    track_ids = list(trajectories.keys())

    for i, track_id1 in enumerate(track_ids):
        fragments = [track_id1]
        traj1 = trajectories[track_id1]

        if len(traj1) < 3:  # Skip very short tracks
            continue

        # Get end time and position of track1
        end_time1 = len(traj1)
        end_pos1 = traj1[-1]

        for j in range(i+1, len(track_ids)):
            track_id2 = track_ids[j]
            traj2 = trajectories[track_id2]

            if len(traj2) < 3:  # Skip very short tracks
                continue

            # Get start time and position of track2
            start_pos2 = traj2[0]

            # Check if track2 starts after track1 ends
            # and check if start position of track2 is close to end position of track1
            distance = np.sqrt((end_pos1[0] - start_pos2[0])**2 + (end_pos1[1] - start_pos2[1])**2)

            # If tracks are close in space and time, they might be fragments
            max_spatial_gap = 50  # pixels
            max_temporal_gap = 10  # frames

            if distance < max_spatial_gap:
                fragments.append(track_id2)

        if len(fragments) > 1:
            potential_fragments.append(fragments)

    # Print fragmentation analysis
    print(f"\nTrack Fragmentation Analysis:")
    print(f"Total tracks: {len(trajectories)}")
    print(f"Potential track fragments: {len(potential_fragments)} groups")

    if potential_fragments:
        print("Fragment groups:")
        for i, group in enumerate(potential_fragments):
            print(f"  Group {i+1}: Track IDs {group}")

    return potential_fragments

# Execution function
def execute_occlusion_tests(model, base_video_path):
    """Execute the occlusion evaluation tests"""

    # 1. Create synthetic occlusion test video
    synthetic_test_path = create_synthetic_occlusion_test(
        base_video_path,
        "synthetic_occlusion_test.mp4"
    )

    # 2. Compare tracking methods on the synthetic test
    comparison_results = compare_tracking_performance(
        model,
        synthetic_test_path,
        max_frames=120
    )

    # 3. Analyze track fragmentation
    fragmentation_analysis = analyze_track_fragmentation(comparison_results['occlusion'])

    # 4. Test on a challenging real video section (if available)
    challenging_section_path = base_video_path  # Use the same video for simplicity

    print("\nTesting on challenging video section...")
    challenging_results = track_with_occlusion_handling(
        challenging_section_path,
        model,
        max_frames=30
    )

    return {
        'synthetic_test': synthetic_test_path,
        'comparison': comparison_results,
        'fragmentation': fragmentation_analysis,
        'challenging': challenging_results
    }

# Main execution function
if __name__ == "__main__":
    # Load model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp3/weights/best.pt')

    # Video path
    video_path = '/kaggle/input/videotraffic/Traffic.mp4'

    # Run tests
    results = execute_occlusion_tests(model, video_path)

    print("\nOcclusion testing complete!")

In [ ]:
# Example usage of the Milestone 4 implementation

# First, let's install the required packages if not already installed
!pip install -q deep-sort-realtime filterpy scipy

# Import necessary libraries
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
import os

# Load the previously trained model
# This path might need to be adjusted based on your environment
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp3/weights/best.pt')

# Define the path to your video
# You can use one of the traffic videos from your dataset
video_path = '/kaggle/input/videotraffic/Traffic.mp4'

# Check if the video exists
if not os.path.exists(video_path):
    print(f"Video not found at {video_path}. Please update the path.")
    # Try to find a traffic video in the current environment
    alternative_paths = [
        '/content/Traffic.mp4',
        '../input/videotraffic/Traffic.mp4',
        './Traffic.mp4'
    ]

    for alt_path in alternative_paths:
        if os.path.exists(alt_path):
            video_path = alt_path
            print(f"Found alternative video at {video_path}")
            break

    if not os.path.exists(video_path):
        print("No suitable video found. Please download a traffic video or adjust the path.")

# Test regular tracking (from Milestone 3)
print("Running standard tracking (Milestone 3)...")
# This assumes you've imported the track_and_predict_behavior function from Milestone 3
standard_results = track_and_predict_behavior(
    video_path,
    model,
    max_frames=20
)

# Test occlusion handling (from Milestone 4)
print("\nRunning occlusion-optimized tracking (Milestone 4)...")
occlusion_results = track_with_occlusion_handling(
    video_path,
    model,
    max_frames=20
)

# Compare results
print("\nComparison of tracking results:")
print(f"Standard tracking: {len(standard_results['trajectories'])} unique vehicles")
print(f"Occlusion-optimized: {len(occlusion_results['trajectories'])} unique vehicles")

# Calculate average track length
if standard_results['trajectories']:
    std_avg_len = sum(len(traj) for traj in standard_results['trajectories'].values()) / len(standard_results['trajectories'])
    print(f"Standard tracking average track length: {std_avg_len:.2f} frames")

if occlusion_results['trajectories']:
    occ_avg_len = sum(len(traj) for traj in occlusion_results['trajectories'].values()) / len(occlusion_results['trajectories'])
    print(f"Occlusion-optimized average track length: {occ_avg_len:.2f} frames")

# Create a synthetic occlusion test
print("\nCreating synthetic occlusion test...")
synthetic_test_path = "synthetic_occlusion_test.mp4"

# Get video dimensions
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

# Create the synthetic test
create_synthetic_occlusion_test(
    video_path,
    synthetic_test_path,
    occlusion_frames=[30, 60, 90],
    occlusion_size=0.3
)

# Run tracking on the synthetic test
print("\nRunning tracking on synthetic occlusion test...")
synthetic_results = track_with_occlusion_handling(
    synthetic_test_path,
    model,
    max_frames=120
)

# Visualize the occlusion handling results
print("\nVisualizing occlusion handling results...")
visualize_occlusion_results(synthetic_results, width, height)

print("\nTesting complete!")

###Milestone 5

In [ ]:
import torch
import cv2
import numpy as np
import time
import os
import threading
import queue
import multiprocessing
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict
import math
from scipy.signal import savgol_filter
import cProfile
import pstats
import io

# Step 11: Implement code profiling
class PerformanceProfiler:
    """Class for profiling code performance"""

    def __init__(self, enabled=True):
        self.enabled = enabled
        self.profiler = None
        self.stats = None
        self.function_timings = defaultdict(float)
        self.section_timings = {}

    def start(self):
        """Start profiling"""
        if self.enabled:
            self.profiler = cProfile.Profile()
            self.profiler.enable()
            return self

    def stop(self):
        """Stop profiling and collect stats"""
        if self.enabled and self.profiler:
            self.profiler.disable()
            s = io.StringIO()
            ps = pstats.Stats(self.profiler, stream=s).sort_stats('cumulative')
            ps.print_stats(30)  # Print top 30 functions by cumulative time
            self.stats = s.getvalue()
            return self.stats

    def start_timer(self, section_name):
        """Start timing a specific section of code"""
        if self.enabled:
            self.section_timings[section_name] = time.time()

    def stop_timer(self, section_name):
        """Stop timing a section and record the elapsed time"""
        if self.enabled and section_name in self.section_timings:
            elapsed = time.time() - self.section_timings[section_name]
            self.function_timings[section_name] += elapsed
            return elapsed
        return 0

    def get_function_timings(self):
        """Get the recorded timings for all functions"""
        return dict(self.function_timings)

    def reset_timings(self):
        """Reset all timing information"""
        self.function_timings = defaultdict(float)
        self.section_timings = {}

    def print_timings(self):
        """Print the timing information in a formatted way"""
        if not self.enabled:
            return

        print("\n--- Function Timing Results ---")
        total_time = sum(self.function_timings.values())

        # Sort by time (descending)
        sorted_times = sorted(self.function_timings.items(), key=lambda x: x[1], reverse=True)

        for func_name, elapsed in sorted_times:
            percentage = (elapsed / total_time * 100) if total_time > 0 else 0
            print(f"{func_name}: {elapsed:.4f}s ({percentage:.1f}%)")

        print(f"Total measured time: {total_time:.4f}s")

    def visualize_timings(self):
        """Create a bar chart of function timings"""
        if not self.enabled or not self.function_timings:
            return

        # Sort by time (descending)
        sorted_times = sorted(self.function_timings.items(), key=lambda x: x[1], reverse=True)
        top_funcs = sorted_times[:10]  # Show top 10 functions

        func_names = [name for name, _ in top_funcs]
        times = [time_val for _, time_val in top_funcs]

        plt.figure(figsize=(12, 6))
        bars = plt.bar(func_names, times, color='skyblue')
        plt.xticks(rotation=45, ha='right')
        plt.title('Top 10 Functions by Execution Time')
        plt.xlabel('Function')
        plt.ylabel('Time (seconds)')
        plt.tight_layout()

        # Add time labels on top of bars
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.3f}s', ha='center', va='bottom', rotation=0)

        plt.show()

# Step 12: Model Optimization Class
class ModelOptimizer:
    """Class for optimizing YOLOv5 models for inference speed"""

    def __init__(self, model_path, device=None):
        self.model_path = model_path
        self.device = device if device else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None
        self.original_model = None

    def load_model(self, quantized=False):
        """Load the YOLOv5 model, with optional quantization"""
        print(f"Loading model from {self.model_path} on {self.device}")

        # Load the original model first
        self.original_model = torch.hub.load('ultralytics/yolov5', 'custom', path=self.model_path)

        # If quantization is requested, apply it
        if quantized and self.device == torch.device("cpu"):
            print("Applying quantization for CPU inference...")
            self.model = self._quantize_model(self.original_model)
        else:
            self.model = self.original_model

        return self.model

    def _quantize_model(self, model):
        """Apply quantization to the model for faster CPU inference"""
        try:
            # Set model to eval mode
            model.eval()

            # Apply static quantization
            quantized_model = torch.quantization.quantize_dynamic(
                model,
                {torch.nn.Linear, torch.nn.Conv2d},
                dtype=torch.qint8
            )

            print("Model quantized successfully")
            return quantized_model
        except Exception as e:
            print(f"Quantization failed: {e}")
            # Return original model if quantization fails
            return model

    def switch_to_smaller_model(self, model_size='yolov5s'):
        """Switch to a smaller YOLOv5 model variant for speed"""
        print(f"Switching to smaller model: {model_size}")
        try:
            # Load a smaller pre-trained model
            smaller_model = torch.hub.load('ultralytics/yolov5', model_size)

            # Transfer the class names from the original model
            if hasattr(self.original_model, 'names'):
                smaller_model.names = self.original_model.names

            self.model = smaller_model
            return self.model
        except Exception as e:
            print(f"Failed to switch to smaller model: {e}")
            return self.model

    def optimize_inference_settings(self, model=None):
        """Optimize model inference settings"""
        if model is None:
            model = self.model

        # Set inference settings
        if model is not None:
            model.conf = 0.25  # Confidence threshold
            model.iou = 0.45   # IoU threshold
            model.classes = None  # Filter by class, None = all classes
            model.agnostic = False  # NMS agnostic (class-independent)
            model.multi_label = False  # Multiple labels per box
            model.max_det = 100  # Maximum detections

        return model

    def benchmark_inference(self, sample_image, iterations=20, warmup=5):
        """Benchmark model inference speed on a sample image"""
        if self.model is None:
            print("No model loaded. Call load_model() first.")
            return

        # Ensure the model is on the right device
        self.model.to(self.device)

        # Warmup runs
        for _ in range(warmup):
            _ = self.model(sample_image)

        # Timed runs
        start_time = time.time()
        for _ in range(iterations):
            _ = self.model(sample_image)

        total_time = time.time() - start_time
        avg_time = total_time / iterations
        fps = iterations / total_time

        print(f"Model inference benchmark:")
        print(f"  Average inference time: {avg_time*1000:.2f} ms")
        print(f"  FPS: {fps:.2f}")

        return {"avg_time": avg_time, "fps": fps}

# Step 13: Multi-threaded Video Processing
class ThreadedVideoProcessor:
    """Class for multi-threaded video processing"""

    def __init__(self, video_path, model, buffer_size=10):
        self.video_path = video_path
        self.model = model
        self.buffer_size = buffer_size

        # Create queues for frame passing between threads
        self.frame_queue = queue.Queue(maxsize=buffer_size)
        self.result_queue = queue.Queue(maxsize=buffer_size)

        # Threading control
        self.running = False
        self.capture_thread = None
        self.detection_thread = None

        # Performance tracking
        self.profiler = PerformanceProfiler()
        self.fps_stats = {"capture": [], "detection": [], "overall": []}
        self.last_frame_time = 0
        self.frame_count = 0

    def start(self):
        """Start the threaded processing"""
        if self.running:
            print("Already running!")
            return

        self.running = True

        # Start the threads
        self.capture_thread = threading.Thread(target=self._capture_worker)
        self.detection_thread = threading.Thread(target=self._detection_worker)

        self.capture_thread.daemon = True
        self.detection_thread.daemon = True

        self.capture_thread.start()
        self.detection_thread.start()

        print("Threaded video processing started")

    def stop(self):
        """Stop the threaded processing"""
        self.running = False

        # Wait for threads to finish
        if self.capture_thread and self.capture_thread.is_alive():
            self.capture_thread.join(timeout=3.0)

        if self.detection_thread and self.detection_thread.is_alive():
            self.detection_thread.join(timeout=3.0)

        # Clear queues
        while not self.frame_queue.empty():
            try:
                self.frame_queue.get_nowait()
            except queue.Empty:
                break

        while not self.result_queue.empty():
            try:
                self.result_queue.get_nowait()
            except queue.Empty:
                break

        print("Threaded video processing stopped")

    def _capture_worker(self):
        """Worker thread for capturing frames from video"""
        cap = cv2.VideoCapture(self.video_path)

        if not cap.isOpened():
            print(f"Error: Could not open video {self.video_path}")
            self.running = False
            return

        frame_count = 0
        fps = cap.get(cv2.CAP_PROP_FPS)

        while self.running:
            self.profiler.start_timer("capture")

            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret:
                print("End of video stream")
                self.running = False
                break

            # Calculate capture FPS
            current_time = time.time()
            if self.last_frame_time > 0:
                capture_fps = 1.0 / (current_time - self.last_frame_time)
                self.fps_stats["capture"].append(capture_fps)
            self.last_frame_time = current_time

            # Put frame in queue, with a timeout to prevent blocking forever
            try:
                self.frame_queue.put((frame_count, frame), timeout=1.0)
                frame_count += 1
            except queue.Full:
                # Skip this frame if queue is full
                pass

            self.profiler.stop_timer("capture")

            # Add a small sleep to prevent CPU overuse
            time.sleep(0.001)

        # Release the video capture object
        cap.release()

    def _detection_worker(self):
        """Worker thread for running object detection"""
        while self.running:
            try:
                # Get a frame from the queue with timeout
                frame_num, frame = self.frame_queue.get(timeout=1.0)

                self.profiler.start_timer("detection")

                # Run detection
                results = self.model(frame)

                # Put the results in the result queue
                detection_time = self.profiler.stop_timer("detection")
                detection_fps = 1.0 / detection_time if detection_time > 0 else 0
                self.fps_stats["detection"].append(detection_fps)

                # Package results and metadata
                result_package = {
                    "frame_num": frame_num,
                    "frame": frame,
                    "results": results,
                    "detection_time": detection_time,
                    "detection_fps": detection_fps
                }

                try:
                    self.result_queue.put(result_package, timeout=1.0)
                except queue.Full:
                    # If the result queue is full, drop the oldest result
                    try:
                        self.result_queue.get_nowait()
                        self.result_queue.put(result_package, timeout=1.0)
                    except (queue.Empty, queue.Full):
                        pass

                # Mark queue task as done
                self.frame_queue.task_done()

            except queue.Empty:
                # No frames available, wait a bit
                time.sleep(0.01)
            except Exception as e:
                print(f"Error in detection worker: {e}")
                continue

    def get_next_result(self, timeout=1.0):
        """Get the next processed frame and detection results"""
        try:
            return self.result_queue.get(timeout=timeout)
        except queue.Empty:
        # Function to run the traffic monitoring system with the optimal configuration
def run_optimized_traffic_monitoring(video_path, model_path, output_path=None, max_frames=None):
    """
    Run the traffic monitoring system with optimized settings

    Args:
        video_path: Path to the input video
        model_path: Path to the YOLOv5 model weights
        output_path: Path to save the output video (optional)
        max_frames: Maximum number of frames to process (optional)

    Returns:
        Performance statistics dictionary
    """
    print("\n=== Running Optimized Traffic Monitoring System ===")

    # First run a quick benchmark to find optimal settings
    print("Running quick benchmark to find optimal configuration...")
    benchmark_result = benchmark_optimized_system(
        video_path=video_path,
        model_path=model_path,
        max_frames=50  # Use a small number of frames for quick benchmarking
    )

    # Extract the best configuration
    use_threading = benchmark_result['threading'] if benchmark_result else True
    use_quantization = benchmark_result['quantization'] if benchmark_result else True

    print(f"\nUsing optimal configuration: Threading={use_threading}, Quantization={use_quantization}")

    # Initialize with the optimal configuration
    processor = OptimizedTrafficProcessor(
        video_path=video_path,
        model_path=model_path,
        output_path=output_path,
        use_threading=use_threading,
        use_quantization=use_quantization
    )

    # Process the video
    print(f"\nProcessing video: {video_path}")
    print(f"Output path: {output_path if output_path else 'None'}")

    start_time = time.time()
    processor.initialize()
    processor.process_video(max_frames=max_frames)
    total_time = time.time() - start_time

    # Calculate performance metrics
    if processor.processing_times:
        avg_processing_time = np.mean(processor.processing_times)
        avg_fps = 1.0 / avg_processing_time if avg_processing_time > 0 else 0
    else:
        avg_processing_time = 0
        avg_fps = 0

    # Return performance statistics
    stats = {
        "total_time": total_time,
        "frames_processed": processor.frame_count,
        "avg_processing_time": avg_processing_time,
        "avg_fps": avg_fps,
        "total_unique_vehicles": processor.total_unique_vehicles,
        "configuration": {
            "threading": use_threading,
            "quantization": use_quantization
        }
    }

    print("\n=== Processing Complete ===")
    print(f"Total time: {total_time:.2f}s")
    print(f"Frames processed: {processor.frame_count}")
    print(f"Average FPS: {avg_fps:.2f}")
    print(f"Unique vehicles detected: {processor.total_unique_vehicles}")

    if output_path:
        print(f"Output video saved to: {output_path}")

    return stats

# Main entry point for demonstration
if __name__ == "__main__":
    import argparse

    # Parse command line arguments
    parser = argparse.ArgumentParser(description="Optimized Traffic Monitoring System")
    parser.add_argument("--video", type=str, required=True, help="Path to input video")
    parser.add_argument("--model", type=str, required=True, help="Path to YOLOv5 model weights")
    parser.add_argument("--output", type=str, default=None, help="Path to save output video")
    parser.add_argument("--max-frames", type=int, default=None, help="Maximum frames to process")
    parser.add_argument("--benchmark-only", action="store_true", help="Run only the benchmark")

    args = parser.parse_args()

    if args.benchmark_only:
        # Run benchmark only
        benchmark_optimized_system(
            video_path=args.video,
            model_path=args.model,
            max_frames=args.max_frames if args.max_frames else 100
        )
    else:
        # Run the full system
        run_optimized_traffic_monitoring(
            video_path=args.video,
            model_path=args.model,
            output_path=args.output,
            max_frames=args.max_frames
        )

    def get_fps_stats(self):
        """Get the FPS statistics"""
        capture_fps = np.mean(self.fps_stats["capture"]) if self.fps_stats["capture"] else 0
        detection_fps = np.mean(self.fps_stats["detection"]) if self.fps_stats["detection"] else 0
        overall_fps = min(capture_fps, detection_fps) if capture_fps > 0 and detection_fps > 0 else 0

        return {
            "capture_fps": capture_fps,
            "detection_fps": detection_fps,
            "overall_fps": overall_fps
        }

# Step 14: Optimized Processing Pipeline
class OptimizedTrafficProcessor:
    """Main class for optimized traffic video processing"""

    def __init__(self, video_path, model_path, output_path=None, use_threading=True, use_quantization=True):
        self.video_path = video_path
        self.model_path = model_path
        self.output_path = output_path
        self.use_threading = use_threading
        self.use_quantization = use_quantization

        # Components initialization
        self.profiler = PerformanceProfiler()
        self.model_optimizer = None
        self.threaded_processor = None
        self.tracker = None
        self.video_writer = None

        # Tracking data
        self.trajectories = defaultdict(list)
        self.speeds = defaultdict(float)
        self.behaviors = defaultdict(str)
        self.violations = defaultdict(bool)

        # Performance metrics
        self.processing_times = []
        self.frame_count = 0
        self.total_vehicles = 0
        self.total_unique_vehicles = 0

        # Visualization settings
        self.colors = np.random.randint(0, 255, size=(100, 3), dtype=np.uint8)

    def initialize(self):
        """Initialize components"""
        # Start profiling
        self.profiler.start()

        # Initialize model optimizer
        self.profiler.start_timer("model_initialization")
        self.model_optimizer = ModelOptimizer(self.model_path)
        model = self.model_optimizer.load_model(quantized=self.use_quantization)
        model = self.model_optimizer.optimize_inference_settings(model)
        self.profiler.stop_timer("model_initialization")

        # Initialize DeepSORT tracker with optimized settings
        self.profiler.start_timer("tracker_initialization")
        self.tracker = DeepSort(
            max_age=30,
            n_init=3,
            nn_budget=100,
            embedder="mobilenet",
            nms_max_overlap=1.0,
            max_cosine_distance=0.2
        )
        self.profiler.stop_timer("tracker_initialization")

        # Initialize threaded processor if enabled
        if self.use_threading:
            self.threaded_processor = ThreadedVideoProcessor(
                self.video_path, model, buffer_size=30
            )

        # Get video properties
        cap = cv2.VideoCapture(self.video_path)
        self.fps = int(cap.get(cv2.CAP_PROP_FPS))
        self.frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        cap.release()

        # Initialize video writer if output path is provided
        if self.output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            self.video_writer = cv2.VideoWriter(
                self.output_path, fourcc, self.fps,
                (self.frame_width, self.frame_height)
            )

        print(f"Initialized with threading={self.use_threading}, quantization={self.use_quantization}")
        print(f"Video properties: {self.frame_width}x{self.frame_height} @ {self.fps} FPS")

        return self

    def process_video(self, max_frames=None):
        """Process the video with the optimized pipeline"""
        if not self.model_optimizer or not self.model_optimizer.model:
            print("Model not initialized. Call initialize() first.")
            return

        model = self.model_optimizer.model

        # Start the threaded processor if enabled
        if self.use_threading and self.threaded_processor:
            self.threaded_processor.start()
            processing_method = self._process_threaded
        else:
            # Open the video file for non-threaded processing
            cap = cv2.VideoCapture(self.video_path)
            if not cap.isOpened():
                print(f"Error: Could not open video {self.video_path}")
                return
            processing_method = lambda: self._process_synchronous(cap, model)

        # Process frames
        try:
            processing_method()

            # Apply max_frames limit if specified
            if max_frames and self.frame_count >= max_frames:
                print(f"Reached maximum frame limit ({max_frames})")

        finally:
            # Clean up resources
            if self.use_threading and self.threaded_processor:
                self.threaded_processor.stop()
            else:
                cap.release()

            if self.video_writer:
                self.video_writer.release()

            # Stop profiling and print results
            self.profiler.stop()
            self.profiler.print_timings()
            self.profiler.visualize_timings()

            # Print summary statistics
            self._print_summary()

        return self

    def _process_threaded(self):
        """Process video using the threaded processor"""
        while True:
            # Get the next processed frame and detection results
            result_package = self.threaded_processor.get_next_result()

            if result_package is None:
                # No more frames or timeout
                if not self.threaded_processor.running:
                    break
                continue

            frame_num = result_package["frame_num"]
            frame = result_package["frame"]
            results = result_package["results"]

            # Process the detection results
            self._process_frame(frame, results)

            # Update frame count
            self.frame_count += 1

    def _process_synchronous(self, cap, model):
        """Process video synchronously (without threading)"""
        while True:
            self.profiler.start_timer("frame_read")
            ret, frame = cap.read()
            frame_read_time = self.profiler.stop_timer("frame_read")

            if not ret:
                print("End of video")
                break

            # Process frame
            self.profiler.start_timer("inference")
            results = model(frame)
            inference_time = self.profiler.stop_timer("inference")

            # Process the detection results
            self._process_frame(frame, results)

            # Update frame count
            self.frame_count += 1

    def _process_frame(self, frame, results):
        """Process a frame and its detection results"""
        # Start timing frame processing
        self.profiler.start_timer("frame_processing")

        # 1. Get detections from results
        self.profiler.start_timer("detections_extraction")
        detections = results.xyxy[0].cpu().numpy()

        # Format detections for DeepSORT
        detection_list = []
        for detection in detections:
            x1, y1, x2, y2, conf, cls = detection
            detection_list.append(
                ([x1, y1, x2 - x1, y2 - y1], conf, int(cls))
            )
        self.profiler.stop_timer("detections_extraction")

        # 2. Update tracker
        self.profiler.start_timer("tracking")
        tracks = self.tracker.update_tracks(detection_list, frame=frame)
        active_tracks = [track for track in tracks if track.is_confirmed()]
        self.profiler.stop_timer("tracking")

        # 3. Update tracking statistics
        frame_vehicle_count = len(active_tracks)
        self.total_vehicles += frame_vehicle_count

        # 4. Process each active track
        self.profiler.start_timer("track_processing")
        processed_frame = self._process_tracks(frame, active_tracks)
        self.profiler.stop_timer("track_processing")

        # 5. Write frame if output is enabled
        if self.video_writer:
            self.profiler.start_timer("frame_write")
            self.video_writer.write(processed_frame)
            self.profiler.stop_timer("frame_write")

        # Calculate processing time for this frame
        frame_processing_time = self.profiler.stop_timer("frame_processing")
        self.processing_times.append(frame_processing_time)

        # Display frame or save for visualization
        self._visualize_frame(processed_frame, frame_processing_time)

        return processed_frame

    def _process_tracks(self, frame, active_tracks):
        """Process all active tracks and draw visualization"""
        # Create a copy of the frame for drawing
        processed_frame = frame.copy()

        # Draw tracking results on the frame
        for track in active_tracks:
            track_id = track.track_id
            bbox = track.to_ltrb()
            x1, y1, x2, y2 = bbox

            # Get track position (center of bounding box)
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            current_pos = (center_x, center_y)

            # Add point to trajectory
            self.trajectories[track_id].append(current_pos)

            # Update speed estimate
            self.speeds[track_id] = self._estimate_speed(self.trajectories[track_id], self.fps)

            # Count unique vehicles
            if len(self.trajectories[track_id]) == 1:
                self.total_unique_vehicles += 1

            # Get color for this track ID
            color = self.colors[track_id % len(self.colors)].tolist()

            # Draw bounding box
            cv2.rectangle(processed_frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

            # Add ID and speed text
            speed_text = f"ID:{track_id} Speed:{self.speeds[track_id]:.1f}"
            cv2.putText(processed_frame, speed_text, (int(x1), int(y1)-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Draw trajectory
            if len(self.trajectories[track_id]) > 1:
                # Draw the trajectory line (last 20 points max for efficiency)
                traj_points = self.trajectories[track_id][-20:]
                for i in range(1, len(traj_points)):
                    pt1 = traj_points[i-1]
                    pt2 = traj_points[i]
                    cv2.line(processed_frame, pt1, pt2, color, 2)

        return processed_frame

    def _estimate_speed(self, traj_points, fps):
        """Estimate vehicle speed based on trajectory points"""
        if len(traj_points) < 5:
            return 0.0

        # Calculate displacement over last 5 points
        distances = []
        for i in range(1, min(6, len(traj_points))):
            distances.append(self._calculate_distance(traj_points[-i], traj_points[-i-1]))

        # Average displacement per frame
        avg_displacement = sum(distances) / len(distances)

        # Convert to speed (pixels per second)
        speed = avg_displacement * fps

        return speed

    def _calculate_distance(self, point1, point2):
        """Calculate Euclidean distance between two points"""
        return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

    def _visualize_frame(self, frame, processing_time):
        """Visualize a processed frame"""
        # Calculate FPS
        fps_current = 1.0 / processing_time if processing_time > 0 else 0

        # Add FPS text to the frame
        cv2.putText(frame, f"FPS: {fps_current:.2f}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Add tracking info
        cv2.putText(frame, f"Vehicles: {len(self.trajectories)}", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Add optimization info
        threading_text = "Threading: ON" if self.use_threading else "Threading: OFF"
        cv2.putText(frame, threading_text, (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Show frame using matplotlib or OpenCV depending on environment
        # This can be customized based on your specific display needs
        if self.frame_count % 10 == 0:  # Show every 10th frame for efficiency
            # Convert BGR to RGB for matplotlib
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            plt.figure(figsize=(12, 8))
            plt.imshow(rgb_frame)
            plt.title(f"Frame {self.frame_count} | FPS: {fps_current:.2f}")
            plt.axis('off')
            plt.show()

    def _print_summary(self):
        """Print summary statistics after processing"""
        avg_processing_time = np.mean(self.processing_times) if self.processing_times else 0
        avg_fps = 1.0 / avg_processing_time if avg_processing_time > 0 else 0

        print("\n--- Processing Summary ---")
        print(f"Total frames processed: {self.frame_count}")
        print(f"Average processing time: {avg_processing_time*1000:.2f} ms per frame")
        print(f"Average FPS: {avg_fps:.2f}")
        print(f"Total unique vehicles: {self.total_unique_vehicles}")
        print(f"Average vehicles per frame: {self.total_vehicles / self.frame_count:.2f}" if self.frame_count else "No frames processed")

        if self.use_threading and self.threaded_processor:
            fps_stats = self.threaded_processor.get_fps_stats()
            print("\n--- Threading Performance ---")
            print(f"Capture FPS: {fps_stats['capture_fps']:.2f}")
            print(f"Detection FPS: {fps_stats['detection_fps']:.2f}")
            print(f"Overall FPS: {fps_stats['overall_fps']:.2f}")

        # Plot FPS over time
        if self.processing_times:
            fps_values = [1.0/t if t > 0 else 0 for t in self.processing_times]

            plt.figure(figsize=(12, 6))
            plt.plot(fps_values, 'b-')
            plt.axhline(y=avg_fps, color='r', linestyle='--', label=f'Average: {avg_fps:.2f} FPS')
            plt.title('Processing FPS over Time')
            plt.xlabel('Frame')
            plt.ylabel('FPS')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()

# Step 15: Benchmarking Function
def benchmark_optimized_system(video_path, model_path, max_frames=100):
    """Benchmark the optimized system with different configurations"""
    print("\n--- Benchmarking Optimized Traffic Monitoring System ---\n")

    configs = [
        {"name": "Baseline (no optimizations)", "threading": False, "quantization": False},
        {"name": "Threading only", "threading": True, "quantization": False},
        {"name": "Quantization only", "threading": False, "quantization": True},
        {"name": "Full optimization", "threading": True, "quantization": True}
    ]

    results = []

    for config in configs:
        print(f"\nTesting configuration: {config['name']}")
        print("-" * 50)

        processor = OptimizedTrafficProcessor(
            video_path=video_path,
            model_path=model_path,
            use_threading=config['threading'],
            use_quantization=config['quantization']
        )

        # Initialize and process
        start_time = time.time()

        try:
            processor.initialize()
            processor.process_video(max_frames=max_frames)

            # Calculate metrics
            processing_time = time.time() - start_time
            avg_processing_time = np.mean(processor.processing_times) if processor.processing_times else 0
            avg_fps = 1.0 / avg_processing_time if avg_processing_time > 0 else 0

            # Store results
            results.append({
                "config": config['name'],
                "threading": config['threading'],
                "quantization": config['quantization'],
                "total_time": processing_time,
                "avg_time_per_frame": avg_processing_time,
                "avg_fps": avg_fps,
                "vehicle_count": processor.total_unique_vehicles
            })

        except Exception as e:
            print(f"Error during benchmark: {e}")

    # Compare results
    print("\n--- Benchmark Results ---")
    print("Configuration | Threading | Quantization | Total Time (s) | Avg FPS")
    print("-" * 75)

    for result in results:
        print(f"{result['config']:<20} | {str(result['threading']):<10} | "
              f"{str(result['quantization']):<12} | {result['total_time']:<14.2f} | {result['avg_fps']:<7.2f}")

    # Plot comparison
    if results:
        plt.figure(figsize=(12, 6))

        # FPS comparison
        plt.subplot(1, 2, 1)
        names = [r['config'] for r in results]
        fps_values = [r['avg_fps'] for r in results]

        bars = plt.bar(names, fps_values, color='skyblue')
        plt.title('Average FPS by Configuration')
        plt.ylabel('Frames Per Second')
        plt.xticks(rotation=45, ha='right')

        # Add FPS values on top of bars
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{height:.2f}', ha='center', va='bottom', rotation=0)

        # Total processing time comparison
        plt.subplot(1, 2, 2)
        times = [r['total_time'] for r in results]

        bars = plt.bar(names, times, color='salmon')
        plt.title('Total Processing Time by Configuration')
        plt.ylabel('Time (seconds)')
        plt.xticks(rotation=45, ha='right')

        # Add time values on top of bars
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{height:.2f}s', ha='center', va='bottom', rotation=0)

        plt.tight_layout()
        plt.show()

    # Return the best configuration based on FPS
    if results:
        best_result = max(results, key=lambda x: x['avg_fps'])
        print(f"\nBest configuration: {best_result['config']} with {best_result['avg_fps']:.2f} FPS")
        return best_result

    return None

In [ ]:
# Import the necessary modules
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import os

# Import our optimized traffic processing system
from milestone5_optimization import (
    PerformanceProfiler,
    ModelOptimizer,
    ThreadedVideoProcessor,
    OptimizedTrafficProcessor,
    benchmark_optimized_system
)

# Define paths
video_path = "/kaggle/input/videotraffic/Traffic.mp4"  # Update this to your video path
model_path = "runs/train/exp3/weights/best.pt"         # Update this to your model path
output_path = "optimized_output.mp4"                   # Output video path

# Example 1: Simple usage with default settings
def example_basic_usage():
    print("Running basic usage example...")

    # Initialize the processor
    processor = OptimizedTrafficProcessor(
        video_path=video_path,
        model_path=model_path,
        output_path=output_path
    )

    # Initialize and process video
    processor.initialize()
    processor.process_video(max_frames=100)  # Process first 100 frames

    print("Basic usage example completed.")

# Example 2: Benchmark different configurations
def example_benchmark():
    print("Running benchmark example...")

    # Run the benchmark with 100 frames
    best_config = benchmark_optimized_system(
        video_path=video_path,
        model_path=model_path,
        max_frames=100
    )

    print(f"Benchmark completed. Best configuration: {best_config}")
    return best_config

# Example 3: Model optimization only
def example_model_optimization():
    print("Running model optimization example...")

    # Initialize the model optimizer
    model_optimizer = ModelOptimizer(model_path)

    # Load the model with quantization
    model = model_optimizer.load_model(quantized=True)

    # Try the smaller model variant
    smaller_model = model_optimizer.switch_to_smaller_model(model_size='yolov5s')

    # Benchmark the models
    print("\nBenchmarking original model:")
    cap = cv2.VideoCapture(video_path)
    ret, sample_frame = cap.read()
    cap.release()

    if ret:
        model_optimizer.benchmark_inference(sample_frame)

        print("\nBenchmarking smaller model:")
        model_optimizer.model = smaller_model
        model_optimizer.benchmark_inference(sample_frame)
    else:
        print("Failed to read a frame from the video for benchmarking.")

    print("Model optimization example completed.")
    return model_optimizer

# Example 4: Profiling only
def example_profiling():
    print("Running profiling example...")

    # Initialize profiler
    profiler = PerformanceProfiler()
    profiler.start()

    # Run some operations to profile
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

    # Time the model inference
    cap = cv2.VideoCapture(video_path)
    frames_to_process = 20

    for i in range(frames_to_process):
        profiler.start_timer(f"frame_{i}")

        ret, frame = cap.read()
        if not ret:
            break

        profiler.start_timer(f"inference_{i}")
        results = model(frame)
        profiler.stop_timer(f"inference_{i}")

        # Do some additional processing
        profiler.start_timer(f"post_processing_{i}")
        detections = results.xyxy[0].cpu().numpy()
        for detection in detections:
            x1, y1, x2, y2, conf, cls = detection
            # Draw bounding boxes (just for profiling purposes)
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        profiler.stop_timer(f"post_processing_{i}")

        profiler.stop_timer(f"frame_{i}")

    cap.release()

    # Stop profiling and print results
    stats = profiler.stop()
    print("\nProfiling results:")
    print(stats)

    # Print and visualize function timings
    profiler.print_timings()
    profiler.visualize_timings()

    print("Profiling example completed.")
    return profiler

# Example 5: Run optimized system with best configuration
def example_run_optimized(use_threading=True, use_quantization=True, max_frames=100):
    print(f"Running optimized system with threading={use_threading}, quantization={use_quantization}...")

    # Initialize the processor with best configuration
    processor = OptimizedTrafficProcessor(
        video_path=video_path,
        model_path=model_path,
        output_path=output_path,
        use_threading=use_threading,
        use_quantization=use_quantization
    )

    # Initialize and process video
    processor.initialize()
    processor.process_video(max_frames=max_frames)

    print("Optimized system run completed.")
    return processor

# Main function to run examples
def main():
    print("Milestone 5: Real-Time Optimization Examples")
    print("=" * 50)

    # Uncomment the examples you want to run

    # Example 1: Basic usage
    # example_basic_usage()

    # Example 2: Benchmark different configurations
    best_config = example_benchmark()

    # Example 3: Model optimization only
    # model_optimizer = example_model_optimization()

    # Example 4: Profiling only
    # profiler = example_profiling()

    # Example 5: Run with best configuration from benchmark
    if best_config:
        example_run_optimized(
            use_threading=best_config.get('threading', True),
            use_quantization=best_config.get('quantization', True),
            max_frames=200
        )
    else:
        # Use default configuration if benchmark wasn't run
        example_run_optimized()

    print("\nAll examples completed.")

if __name__ == "__main__":
    main()

###Final Version

In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import threading
import queue
import math
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict
from scipy.signal import savgol_filter
import argparse
import sys

# Global configuration
CONFIG = {
    "use_threading": True,           # Use multi-threading for improved performance
    "use_quantization": True,        # Use model quantization for faster CPU inference
    "display_results": True,         # Show visualizations during processing
    "save_output": True,             # Save output video
    "show_every_nth_frame": 10,      # Show visualization every Nth frame (higher = faster)
    "confidence_threshold": 0.3,     # Detection confidence threshold
    "max_frames": None,              # Maximum frames to process (None = all frames)
    "enable_behavior_prediction": True,  # Enable behavior prediction (Milestone 3)
    "enable_occlusion_handling": True,   # Enable occlusion handling (Milestone 4)
    "enable_profiling": False,        # Enable detailed performance profiling
}

class PerformanceTracker:
    """Simple performance tracking for timing code sections"""

    def __init__(self):
        self.timings = defaultdict(list)
        self.start_times = {}

    def start(self, section_name):
        """Start timing a section"""
        self.start_times[section_name] = time.time()

    def stop(self, section_name):
        """Stop timing a section and record the elapsed time"""
        if section_name in self.start_times:
            elapsed = time.time() - self.start_times[section_name]
            self.timings[section_name].append(elapsed)
            del self.start_times[section_name]
            return elapsed
        return 0

    def get_avg_timing(self, section_name):
        """Get average timing for a section"""
        if section_name in self.timings and self.timings[section_name]:
            return sum(self.timings[section_name]) / len(self.timings[section_name])
        return 0

    def print_summary(self):
        """Print timing summary"""
        print("\n----- Performance Summary -----")
        for section, times in self.timings.items():
            if times:
                avg_time = sum(times) / len(times)
                max_time = max(times)
                min_time = min(times)
                print(f"{section}: avg={avg_time*1000:.2f}ms, min={min_time*1000:.2f}ms, max={max_time*1000:.2f}ms")

class ThreadedVideoProcessor:
    """Multi-threaded video processor for improved performance"""

    def __init__(self, video_path, model, buffer_size=10):
        self.video_path = video_path
        self.model = model
        self.buffer_size = buffer_size

        # Create queues for frame passing between threads
        self.frame_queue = queue.Queue(maxsize=buffer_size)
        self.result_queue = queue.Queue(maxsize=buffer_size)

        # Threading control
        self.running = False
        self.capture_thread = None
        self.detection_thread = None

        # Video properties
        self.cap = cv2.VideoCapture(video_path)
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))
        self.frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.cap.release()  # Release for now, will reopen in thread

        # Performance tracking
        self.performance = PerformanceTracker()

    def start(self):
        """Start the threaded processing"""
        if self.running:
            print("Already running!")
            return

        self.running = True

        # Start the threads
        self.capture_thread = threading.Thread(target=self._capture_worker)
        self.detection_thread = threading.Thread(target=self._detection_worker)

        self.capture_thread.daemon = True
        self.detection_thread.daemon = True

        self.capture_thread.start()
        self.detection_thread.start()

        print("Threaded video processing started")

    def stop(self):
        """Stop the threaded processing"""
        self.running = False

        # Wait for threads to finish
        if self.capture_thread and self.capture_thread.is_alive():
            self.capture_thread.join(timeout=3.0)

        if self.detection_thread and self.detection_thread.is_alive():
            self.detection_thread.join(timeout=3.0)

        # Clear queues
        while not self.frame_queue.empty():
            try:
                self.frame_queue.get_nowait()
            except queue.Empty:
                break

        while not self.result_queue.empty():
            try:
                self.result_queue.get_nowait()
            except queue.Empty:
                break

        print("Threaded video processing stopped")

    def _capture_worker(self):
        """Worker thread for capturing frames from video"""
        cap = cv2.VideoCapture(self.video_path)

        if not cap.isOpened():
            print(f"Error: Could not open video {self.video_path}")
            self.running = False
            return

        frame_count = 0

        while self.running:
            self.performance.start("capture")

            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret:
                print("End of video stream")
                self.running = False
                break

            # Put frame in queue, with a timeout to prevent blocking forever
            try:
                self.frame_queue.put((frame_count, frame), timeout=1.0)
                frame_count += 1
            except queue.Full:
                # Skip this frame if queue is full
                pass

            self.performance.stop("capture")

            # Add a small sleep to prevent CPU overuse
            time.sleep(0.001)

        # Release the video capture object
        cap.release()

    def _detection_worker(self):
        """Worker thread for running object detection"""
        while self.running:
            try:
                # Get a frame from the queue with timeout
                frame_num, frame = self.frame_queue.get(timeout=1.0)

                self.performance.start("detection")

                # Run detection
                results = self.model(frame)

                # Calculate detection time and FPS
                detection_time = self.performance.stop("detection")
                detection_fps = 1.0 / detection_time if detection_time > 0 else 0

                # Package results and metadata
                result_package = {
                    "frame_num": frame_num,
                    "frame": frame.copy(),  # Create a copy to avoid reference issues
                    "results": results,
                    "detection_time": detection_time,
                    "detection_fps": detection_fps
                }

                try:
                    self.result_queue.put(result_package, timeout=1.0)
                except queue.Full:
                    # If the result queue is full, drop the oldest result
                    try:
                        self.result_queue.get_nowait()
                        self.result_queue.put(result_package, timeout=1.0)
                    except (queue.Empty, queue.Full):
                        pass

                # Mark queue task as done
                self.frame_queue.task_done()

            except queue.Empty:
                # No frames available, wait a bit
                time.sleep(0.01)
            except Exception as e:
                print(f"Error in detection worker: {e}")
                continue

    def get_next_result(self, timeout=1.0):
        """Get the next processed frame and detection results"""
        try:
            return self.result_queue.get(timeout=timeout)
        except queue.Empty:
            return None

class TrafficMonitoringSystem:
    """Integrated traffic monitoring system with all milestones and real-time optimization"""

    def __init__(self, config=None):
        # Update configuration with custom settings
        self.config = CONFIG.copy()
        if config:
            self.config.update(config)

        # Initialize components
        self.performance = PerformanceTracker()
        self.model = None
        self.threaded_processor = None
        self.tracker = None
        self.video_writer = None

        # Tracking data
        self.trajectories = defaultdict(list)
        self.smooth_trajectories = defaultdict(list)
        self.speeds = defaultdict(float)
        self.behaviors = defaultdict(str)
        self.violations = defaultdict(bool)
        self.is_visible = defaultdict(bool)
        self.occlusion_stats = defaultdict(lambda: {"count": 0, "total_frames": 0, "current_streak": 0})

        # Performance metrics
        self.processing_times = []
        self.frame_count = 0
        self.total_vehicles = 0
        self.total_unique_vehicles = 0
        self.total_violations = 0

        # Visualization settings
        self.colors = np.random.randint(0, 255, size=(100, 3), dtype=np.uint8)

    def load_model(self, model_path):
        """Load the YOLOv5 model with optimizations"""
        print(f"Loading model from {model_path}")
        self.performance.start("model_loading")

        # Load the model
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

        # Apply optimizations
        self.model.conf = self.config["confidence_threshold"]  # Confidence threshold
        self.model.iou = 0.45  # IoU threshold
        self.model.to(device)

        # Apply quantization if enabled and using CPU
        if self.config["use_quantization"] and device.type == 'cpu':
            print("Applying quantization for CPU inference...")
            try:
                # Set model to eval mode
                self.model.eval()

                # Apply static quantization to applicable parts
                quantized_model = torch.quantization.quantize_dynamic(
                    self.model,
                    {torch.nn.Linear, torch.nn.Conv2d},
                    dtype=torch.qint8
                )

                self.model = quantized_model
                print("Model quantized successfully")
            except Exception as e:
                print(f"Quantization failed: {e}")

        load_time = self.performance.stop("model_loading")
        print(f"Model loaded in {load_time:.2f}s")
        return self.model

    def initialize_tracker(self):
        """Initialize the DeepSORT tracker with optimized settings"""
        self.tracker = DeepSort(
            max_age=30,               # Maximum frames to keep track
            n_init=3,                 # Frames before a track is confirmed
            nn_budget=100,            # Maximum appearance features to store
            embedder="mobilenet",     # Feature extractor
            nms_max_overlap=0.8 if self.config["enable_occlusion_handling"] else 1.0,
            max_cosine_distance=0.2,
            max_iou_distance=0.7 if self.config["enable_occlusion_handling"] else 0.7
        )
        return self.tracker

    def initialize_video_processor(self, video_path):
        """Initialize the video processor (threaded or synchronous)"""
        if self.config["use_threading"]:
            self.threaded_processor = ThreadedVideoProcessor(
                video_path, self.model, buffer_size=30
            )
            self.threaded_processor.start()
        else:
            # Just open the video for synchronous processing
            self.cap = cv2.VideoCapture(video_path)
            if not self.cap.isOpened():
                print(f"Error: Could not open video {video_path}")
                return False

            # Get video properties
            self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))
            self.frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            self.frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        return True

    def initialize_video_writer(self, output_path):
        """Initialize video writer if saving output"""
        if self.config["save_output"]:
            # Use video properties from threaded processor or direct capture
            if self.config["use_threading"]:
                fps = self.threaded_processor.fps
                frame_width = self.threaded_processor.frame_width
                frame_height = self.threaded_processor.frame_height
            else:
                fps = self.fps
                frame_width = self.frame_width
                frame_height = self.frame_height

            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            self.video_writer = cv2.VideoWriter(
                output_path, fourcc, fps, (frame_width, frame_height)
            )
            print(f"Video output will be saved to {output_path}")

        return self.video_writer is not None if self.config["save_output"] else True

    def initialize(self, video_path, model_path, output_path=None):
        """Initialize all components of the system"""
        print("Initializing Traffic Monitoring System...")

        # Load model
        self.load_model(model_path)

        # Initialize tracker
        self.initialize_tracker()

        # Initialize video processor
        if not self.initialize_video_processor(video_path):
            return False

        # Initialize video writer if output path is provided
        if output_path and self.config["save_output"]:
            self.initialize_video_writer(output_path)

        print("Initialization complete")
        return True

    def process_video(self):
        """Process the entire video"""
        # Start the performance tracking
        self.performance.start("total_processing")

        if self.config["use_threading"]:
            self._process_threaded()
        else:
            self._process_synchronous()

        # Cleanup
        if self.video_writer:
            self.video_writer.release()

        if not self.config["use_threading"] and hasattr(self, 'cap'):
            self.cap.release()

        # Print performance summary
        total_time = self.performance.stop("total_processing")
        print(f"\nProcessed {self.frame_count} frames in {total_time:.2f}s")
        print(f"Average FPS: {self.frame_count / total_time:.2f}")
        print(f"Total unique vehicles: {self.total_unique_vehicles}")
        print(f"Total violations detected: {self.total_violations}")

        if self.config["enable_profiling"]:
            self.performance.print_summary()

        return {
            "frames_processed": self.frame_count,
            "total_time": total_time,
            "avg_fps": self.frame_count / total_time if total_time > 0 else 0,
            "unique_vehicles": self.total_unique_vehicles,
            "violations": self.total_violations
        }

    def _process_threaded(self):
        """Process video using threaded pipeline"""
        frame_index = 0

        while True:
            # Check if we've reached the maximum frames
            if self.config["max_frames"] and frame_index >= self.config["max_frames"]:
                print(f"Reached maximum frame limit ({self.config['max_frames']})")
                break

            # Get the next result from the threaded processor
            result_package = self.threaded_processor.get_next_result()

            if result_package is None:
                # No more frames or timeout
                if not self.threaded_processor.running:
                    break
                continue

            frame = result_package["frame"]
            results = result_package["results"]
            detection_fps = result_package["detection_fps"]

            # Process the detection results
            self.performance.start("tracking")
            processed_frame = self._process_detections(frame, results)
            tracking_time = self.performance.stop("tracking")

            # Calculate overall FPS
            overall_fps = 1.0 / (tracking_time + result_package["detection_time"])

            # Add FPS info to the frame
            cv2.putText(
                processed_frame,
                f"FPS: {overall_fps:.1f} | Vehicles: {len(self.trajectories)} | Violations: {self.total_violations}",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2
            )

            # Save frame to video if enabled
            if self.video_writer:
                self.video_writer.write(processed_frame)

            # Display the frame if visualization is enabled
            if self.config["display_results"] and frame_index % self.config["show_every_nth_frame"] == 0:
                self._display_frame(processed_frame, frame_index, overall_fps)

            # Update frame count
            self.frame_count += 1
            frame_index += 1

        # Stop the threaded processor
        self.threaded_processor.stop()

    def _process_synchronous(self):
        """Process video synchronously (without threading)"""
        frame_index = 0

        while True:
            # Check if we've reached the maximum frames
            if self.config["max_frames"] and frame_index >= self.config["max_frames"]:
                print(f"Reached maximum frame limit ({self.config['max_frames']})")
                break

            # Start timing this frame
            self.performance.start("frame")

            # Read the next frame
            self.performance.start("read")
            ret, frame = self.cap.read()
            read_time = self.performance.stop("read")

            if not ret:
                print("End of video")
                break

            # Run detection
            self.performance.start("detection")
            results = self.model(frame)
            detection_time = self.performance.stop("detection")

            # Process detections and update tracking
            self.performance.start("tracking")
            processed_frame = self._process_detections(frame, results)
            tracking_time = self.performance.stop("tracking")

            # Calculate FPS
            frame_time = self.performance.stop("frame")
            fps = 1.0 / frame_time if frame_time > 0 else 0

            # Add FPS info to the frame
            cv2.putText(
                processed_frame,
                f"FPS: {fps:.1f} | Vehicles: {len(self.trajectories)} | Violations: {self.total_violations}",
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2
            )

            # Save frame to video if enabled
            if self.video_writer:
                self.video_writer.write(processed_frame)

            # Display the frame if visualization is enabled
            if self.config["display_results"] and frame_index % self.config["show_every_nth_frame"] == 0:
                self._display_frame(processed_frame, frame_index, fps)

            # Update frame count
            self.frame_count += 1
            frame_index += 1

            # Store processing time for statistics
            self.processing_times.append(frame_time)

    def _process_detections(self, frame, results):
        """Process detection results and update tracking"""
        # Get detections
        detections = results.xyxy[0].cpu().numpy()

        # Format detections for DeepSORT
        detection_list = []
        for detection in detections:
            x1, y1, x2, y2, conf, cls = detection
            detection_list.append(
                ([x1, y1, x2 - x1, y2 - y1], conf, int(cls))
            )

        # Update tracker
        tracks = self.tracker.update_tracks(detection_list, frame=frame)
        active_tracks = [track for track in tracks if track.is_confirmed()]

        # Count active tracks in this frame
        frame_vehicle_count = len(active_tracks)
        self.total_vehicles += frame_vehicle_count

        # Create a copy of the frame for visualization
        tracked_frame = frame.copy()

        # Define intersection zones for behavior prediction (Milestone 3)
        if self.config["enable_behavior_prediction"]:
            # Simple intersection zone definition - adjust based on your video
            intersection = self._define_intersection_zones(frame.shape[1], frame.shape[0])
            self._draw_intersection_zones(tracked_frame, intersection)

        # Update visibility status for all tracks
        current_track_ids = set()

        # Track IDs that were visible in previous frame but not in current frame
        if self.config["enable_occlusion_handling"]:
            previously_visible = set(track_id for track_id, visible in self.is_visible.items() if visible)

        # Process each active track
        for track in active_tracks:
            track_id = track.track_id
            current_track_ids.add(track_id)

            # Mark track as visible
            if self.config["enable_occlusion_handling"]:
                self.is_visible[track_id] = True
                # Reset occlusion streak counter
                self.occlusion_stats[track_id]["current_streak"] = 0

            bbox = track.to_ltrb()
            x1, y1, x2, y2 = bbox

            # Get track position (center of bounding box)
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            current_pos = (center_x, center_y)

            # Add point to trajectory
            self.trajectories[track_id].append(current_pos)

            # Update smooth trajectory if occlusion handling is enabled
            if self.config["enable_occlusion_handling"] and len(self.trajectories[track_id]) > 5:
                self.smooth_trajectories[track_id] = self._smooth_trajectory(self.trajectories[track_id])
            else:
                self.smooth_trajectories[track_id] = self.trajectories[track_id].copy()

            # Update speed estimate
            if hasattr(self, 'fps'):
                self.speeds[track_id] = self._estimate_speed(self.trajectories[track_id], self.fps)

            # Count unique vehicles
            if len(self.trajectories[track_id]) == 1:
                self.total_unique_vehicles += 1

            # Analyze behavior if enabled (Milestone 3)
            if self.config["enable_behavior_prediction"] and len(self.trajectories[track_id]) >= 10:
                # Determine which region the vehicle is in
                if center_x < intersection['center'][0] and center_y < intersection['center'][1]:
                    approach_direction = 'north'
                elif center_x >= intersection['center'][0] and center_y < intersection['center'][1]:
                    approach_direction = 'east'
                elif center_x < intersection['center'][0] and center_y >= intersection['center'][1]:
                    approach_direction = 'west'
                else:
                    approach_direction = 'south'

                # Update behavior
                self.behaviors[track_id] = self._analyze_trajectory_direction(self.trajectories[track_id])

                # Check for red light violation
                stop_line_y = intersection['stop_lines'][approach_direction]
                light_state = intersection['light_states'][approach_direction]

                # Detect violation
                new_violation = self._detect_red_light_violation(
                    self.trajectories[track_id],
                    current_pos,
                    stop_line_y,
                    light_state
                )

                # Count new violations
                if new_violation and not self.violations[track_id]:
                    self.total_violations += 1

                self.violations[track_id] = new_violation

            # Get color for this track ID
            color = self.colors[track_id % len(self.colors)].tolist()

            # Draw bounding box (red for violation, normal color otherwise)
            box_color = (0, 0, 255) if self.violations.get(track_id, False) else color
            cv2.rectangle(tracked_frame, (int(x1), int(y1)), (int(x2), int(y2)), box_color, 2)

            # Add ID and info text
            info_text = f"ID:{track_id}"
            if self.config["enable_behavior_prediction"]:
                info_text += f" {self.behaviors.get(track_id, '')}"
            cv2.putText(tracked_frame, info_text, (int(x1), int(y1)-10),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Add speed text
            cv2.putText(tracked_frame, f"Speed:{self.speeds.get(track_id, 0):.1f}", (int(x1), int(y1)-30),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Add violation warning
            if self.violations.get(track_id, False):
                cv2.putText(tracked_frame, "VIOLATION!", (int(x1), int(y1)-50),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Draw trajectory
            if len(self.smooth_trajectories[track_id]) > 1:
                # Draw the trajectory line (last 20 points max for efficiency)
                traj_points = self.smooth_trajectories[track_id][-20:]
                for i in range(1, len(traj_points)):
                    pt1 = tuple(map(int, traj_points[i-1]))
                    pt2 = tuple(map(int, traj_points[i]))
                    cv2.line(tracked_frame, pt1, pt2, color, 2)

        # Handle occlusions if enabled (Milestone 4)
        if self.config["enable_occlusion_handling"]:
            # Find tracks that are occluded in current frame
            all_known_tracks = set(self.trajectories.keys())
            occluded_tracks = all_known_tracks - current_track_ids

            for track_id in occluded_tracks:
                # Skip if track has been missing for too long
                if self.frame_count - len(self.trajectories[track_id]) > 30:  # max_age
                    continue

                # Track is occluded in this frame
                if self.is_visible[track_id]:  # First frame of occlusion
                    self.is_visible[track_id] = False
                    self.occlusion_stats[track_id]["count"] += 1

                # Update occlusion streak counter
                self.occlusion_stats[track_id]["current_streak"] += 1
                self.occlusion_stats[track_id]["total_frames"] += 1

                # Predict position during occlusion
                if len(self.trajectories[track_id]) > 5:
                    # Simple linear prediction based on last few positions
                    last_positions = self.trajectories[track_id][-5:]
                    dx = np.mean([last_positions[i][0] - last_positions[i-1][0] for i in range(1, 5)])
                    dy = np.mean([last_positions[i][1] - last_positions[i-1][1] for i in range(1, 5)])

                    last_pos = last_positions[-1]
                    predicted_pos = (int(last_pos[0] + dx), int(last_pos[1] + dy))

                    # Add predicted position to trajectory
                    self.trajectories[track_id].append(predicted_pos)

                    # Update smoothed trajectory
                    self.smooth_trajectories[track_id] = self._smooth_trajectory(self.trajectories[track_id])

                    # Draw occluded bounding box with dashed lines
                    box_width, box_height = 60, 60  # Approximate size
                    cv2.rectangle(
                        tracked_frame,
                        (predicted_pos[0] - box_width//2, predicted_pos[1] - box_height//2),
                        (predicted_pos[0] + box_width//2, predicted_pos[1] + box_height//2),
                        (255, 255, 0), 1, cv2.LINE_DASHED
                    )

                    # Draw text
                    cv2.putText(
                        tracked_frame,
                        f"Occluded ID:{track_id}",
                        (predicted_pos[0] - box_width//2, predicted_pos[1] - box_height//2 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2
                    )

        return tracked_frame

    def _display_frame(self, frame, frame_index, fps):
        """Display a processed frame"""
        # Convert BGR to RGB for matplotlib
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        plt.figure(figsize=(12, 8))
        plt.imshow(frame_rgb)
        plt.title(f"Frame {frame_index} | FPS: {fps:.2f} | Vehicles: {self.total_unique_vehicles} | Violations: {self.total_violations}")
        plt.axis('off')
        plt.tight_layout()
        plt.show()

    # Helper functions below

    def _estimate_speed(self, traj_points, fps):
        """Estimate vehicle speed based on trajectory points"""
        if len(traj_points) < 5:
            return 0.0

        # Calculate displacement over last 5 points
        distances = []
        for i in range(1, min(6, len(traj_points))):
            distances.append(self._calculate_distance(traj_points[-i], traj_points[-i-1]))

        # Average displacement per frame
        avg_displacement = sum(distances) / len(distances)

        # Convert to speed (pixels per second)
        speed = avg_displacement * fps

        return speed

    def _calculate_distance(self, point1, point2):
        """Calculate Euclidean distance between two points"""
        return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

    def _smooth_trajectory(self, trajectory, window_size=15, poly_order=3):
        """Smooth a trajectory using Savitzky-Golay filter"""
        if len(trajectory) < window_size:
            return trajectory

        # Convert to numpy array
        traj_np = np.array(trajectory)

        # Adjust window size if needed
        if window_size % 2 == 0:
            window_size -= 1

        if window_size > len(traj_np):
            window_size = len(traj_np) - 1
            if window_size % 2 == 0:
                window_size -= 1

        if window_size < poly_order + 1:
            return trajectory

        try:
            # Smooth x and y separately
            x_smooth = savgol_filter(traj_np[:, 0], window_size, poly_order)
            y_smooth = savgol_filter(traj_np[:, 1], window_size, poly_order)

            # Combine back to pairs
            smoothed_traj = list(zip(x_smooth, y_smooth))
            return smoothed_traj
        except Exception as e:
            print(f"Smoothing error: {e}")
            return trajectory

    def _analyze_trajectory_direction(self, trajectory, window_size=10):
        """Analyze trajectory to determine if vehicle is turning and in which direction"""
        if len(trajectory) < window_size:
            return "Unknown"

        # Get recent trajectory points
        recent_traj = trajectory[-window_size:]

        # Calculate direction changes
        angles = []
        for i in range(1, len(recent_traj)):
            dx = recent_traj[i][0] - recent_traj[i-1][0]
            dy = recent_traj[i][1] - recent_traj[i-1][1]
            angle = math.atan2(dy, dx)
            angles.append(angle)

        # Calculate angle difference between start and end
        angle_diff = abs(angles[-1] - angles[0])

        # Classify turn direction
        if angle_diff > 0.3:  # Threshold for turn detection (in radians)
            # Determine turn direction
            if (angles[-1] - angles[0]) > 0:
                return "Turning Left"
            else:
                return "Turning Right"
        else:
            return "Going Straight"

    def _detect_red_light_violation(self, trajectory, current_pos, stop_line_y, light_state):
        """Detect if a vehicle might run a red light"""
        if light_state != "RED" or len(trajectory) < 5:
            return False

        # Check if vehicle is approaching the stop line
        if current_pos[1] < stop_line_y and current_pos[1] > stop_line_y - 100:
            # Calculate speed
            recent_traj = trajectory[-5:]
            distances = []
            for i in range(1, len(recent_traj)):
                distances.append(self._calculate_distance(recent_traj[i], recent_traj[i-1]))

            avg_speed = sum(distances) / len(distances)

            # If approaching fast and not slowing down
            if avg_speed > 5.0:  # Threshold for "fast" approach
                return True

        return False

    def _define_intersection_zones(self, frame_width, frame_height):
        """Define virtual zones for traffic analysis based on frame dimensions"""
        # Example for a four-way intersection
        intersection = {
            'center': (frame_width//2, frame_height//2),
            'radius': 150,
            'stop_lines': {
                'north': frame_height//2 - 100,
                'south': frame_height//2 + 100,
                'east': frame_width//2 + 100,
                'west': frame_width//2 - 100
            },
            'light_states': {
                'north': 'RED',
                'south': 'RED',
                'east': 'GREEN',
                'west': 'RED'
            }
        }

        return intersection

    def _draw_intersection_zones(self, frame, intersection):
        """Draw intersection zones on the frame"""
        frame_width = frame.shape[1]
        frame_height = frame.shape[0]

        # Draw center circle
        cv2.circle(frame, intersection['center'], 10, (0, 0, 255), -1)
        cv2.circle(frame, intersection['center'], intersection['radius'], (0, 0, 255), 2)

        # Draw stop lines
        cv2.line(frame,
                 (0, intersection['stop_lines']['north']),
                 (frame_width, intersection['stop_lines']['north']),
                 (255, 0, 0), 2)
        cv2.line(frame,
                 (0, intersection['stop_lines']['south']),
                 (frame_width, intersection['stop_lines']['south']),
                 (255, 0, 0), 2)
        cv2.line(frame,
                 (intersection['stop_lines']['west'], 0),
                 (intersection['stop_lines']['west'], frame_height),
                 (255, 0, 0), 2)
        cv2.line(frame,
                 (intersection['stop_lines']['east'], 0),
                 (intersection['stop_lines']['east'], frame_height),
                 (255, 0, 0), 2)

        # Add traffic light indicators
        light_positions = {
            'north': (frame_width//2 - 50, intersection['stop_lines']['north'] - 20),
            'south': (frame_width//2 + 50, intersection['stop_lines']['south'] + 20),
            'east': (intersection['stop_lines']['east'] + 20, frame_height//2 - 50),
            'west': (intersection['stop_lines']['west'] - 20, frame_height//2 + 50)
        }

        for direction, position in light_positions.items():
            # Determine light color
            if intersection['light_states'][direction] == 'RED':
                light_color = (0, 0, 255)  # Red
            elif intersection['light_states'][direction] == 'YELLOW':
                light_color = (0, 255, 255)  # Yellow
            else:  # GREEN
                light_color = (0, 255, 0)  # Green

            # Draw light indicator
            cv2.circle(frame, position, 15, light_color, -1)
            cv2.circle(frame, position, 15, (255, 255, 255), 2)

        return frame

    def _update_traffic_lights(self, intersection, frame_count, cycle_length=150):
        """Update traffic light states based on a simple cycle"""
        # Simple traffic light cycle: N/S green, E/W red, then switch
        cycle_position = frame_count % cycle_length

        # First half of cycle: N/S green, E/W red
        if cycle_position < cycle_length // 2:
            intersection['light_states']['north'] = 'GREEN'
            intersection['light_states']['south'] = 'GREEN'
            intersection['light_states']['east'] = 'RED'
            intersection['light_states']['west'] = 'RED'

            # Add yellow transition
            if cycle_position > (cycle_length // 2) - 30:
                intersection['light_states']['north'] = 'YELLOW'
                intersection['light_states']['south'] = 'YELLOW'
        # Second half: N/S red, E/W green
        else:
            intersection['light_states']['north'] = 'RED'
            intersection['light_states']['south'] = 'RED'
            intersection['light_states']['east'] = 'GREEN'
            intersection['light_states']['west'] = 'GREEN'

            # Add yellow transition
            if cycle_position > cycle_length - 30:
                intersection['light_states']['east'] = 'YELLOW'
                intersection['light_states']['west'] = 'YELLOW'

        return intersection

# Main function to run the traffic monitoring system
def run_traffic_monitoring(
    video_path,
    model_path,
    output_path=None,
    use_threading=True,
    use_quantization=True,
    max_frames=None,
    show_visualization=True
):
    """
    Run the complete traffic monitoring system with all optimizations.

    Args:
        video_path: Path to the input video
        model_path: Path to the YOLOv5 model weights
        output_path: Path to save the output video (None = no output)
        use_threading: Enable multi-threading for better performance
        use_quantization: Enable model quantization for faster CPU inference
        max_frames: Maximum number of frames to process (None = all frames)
        show_visualization: Whether to display visualizations

    Returns:
        Dictionary with processing statistics
    """
    # Configure the system
    config = {
        "use_threading": use_threading,
        "use_quantization": use_quantization,
        "display_results": show_visualization,
        "save_output": output_path is not None,
        "max_frames": max_frames,
        "enable_behavior_prediction": True,
        "enable_occlusion_handling": True,
        "enable_profiling": True
    }

    # Initialize the system
    system = TrafficMonitoringSystem(config)

    # Start the system
    print(f"Starting traffic monitoring on {video_path}")
    print(f"Model: {model_path}")
    print(f"Configuration: threading={use_threading}, quantization={use_quantization}")

    if not system.initialize(video_path, model_path, output_path):
        print("Initialization failed.")
        return None

    # Process the video
    results = system.process_video()

    print("\nProcessing complete!")
    if output_path and config["save_output"]:
        print(f"Output video saved to: {output_path}")

    return results

# Command-line interface
def main():
    parser = argparse.ArgumentParser(description="Optimized Traffic Monitoring System")
    parser.add_argument("--video", type=str, required=True, help="Path to input video")
    parser.add_argument("--model", type=str, required=True, help="Path to YOLOv5 model weights")
    parser.add_argument("--output", type=str, default=None, help="Path to save output video")
    parser.add_argument("--no-threading", action="store_true", help="Disable multi-threading")
    parser.add_argument("--no-quantization", action="store_true", help="Disable model quantization")
    parser.add_argument("--max-frames", type=int, default=None, help="Maximum frames to process")
    parser.add_argument("--no-visualization", action="store_true", help="Disable visualization")

    args = parser.parse_args()

    # Run the system
    run_traffic_monitoring(
        video_path=args.video,
        model_path=args.model,
        output_path=args.output,
        use_threading=not args.no_threading,
        use_quantization=not args.no_quantization,
        max_frames=args.max_frames,
        show_visualization=not args.no_visualization
    )

if __name__ == "__main__":
    main()

In [ ]:
# Run Traffic Monitoring Demo
# This script demonstrates the full traffic monitoring system with real-time optimization

import os
import torch
import time
import matplotlib.pyplot as plt
import numpy as np

# Import our system
from traffic_monitoring_final import TrafficMonitoringSystem, run_traffic_monitoring

# Ensure the required packages are installed
try:
    import deep_sort_realtime
except ImportError:
    print("Installing deep-sort-realtime...")
    !pip install -q deep-sort-realtime

try:
    from scipy.signal import savgol_filter
except ImportError:
    print("Installing scipy...")
    !pip install -q scipy

# Set the paths to your files
# Adjust these paths to match your environment
VIDEO_PATH = "/root/.cache/kagglehub/datasets/raguhudarare/videotraffic/versions/2/Traffic.mp4"
MODEL_PATH = "runs/train/exp/weights/best.pt"  # Path to your trained model
OUTPUT_PATH = "traffic_monitoring_output.mp4"

# Check if video exists
if not os.path.exists(VIDEO_PATH):
    print(f"Video not found at {VIDEO_PATH}")
    print("Searching for alternative video paths...")

    # Try some common alternatives
    alternatives = [
        "/kaggle/input/videotraffic/Traffic.mp4",
        "../input/videotraffic/Traffic.mp4",
        "Traffic.mp4"
    ]

    for alt_path in alternatives:
        if os.path.exists(alt_path):
            VIDEO_PATH = alt_path
            print(f"Found video at {VIDEO_PATH}")
            break
    else:
        print("No suitable video found. Please update the VIDEO_PATH variable.")

# Check if model exists
if not os.path.exists(MODEL_PATH):
    print(f"Model not found at {MODEL_PATH}")
    print("Searching for alternative model paths...")

    # Try some common alternatives
    alternatives = [
        "runs/train/exp3/weights/best.pt",
        "/kaggle/working/yolov5/runs/train/exp/weights/best.pt",
        "yolov5s.pt"  # Fall back to pre-trained model
    ]

    for alt_path in alternatives:
        if os.path.exists(alt_path):
            MODEL_PATH = alt_path
            print(f"Found model at {MODEL_PATH}")
            break
    else:
        print("No suitable model found. Will try to download YOLOv5s...")
        try:
            # Try to download the pre-trained YOLOv5s model
            model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
            MODEL_PATH = "yolov5s.pt"
            print("Downloaded pre-trained YOLOv5s model")
        except Exception as e:
            print(f"Failed to download model: {e}")
            print("Please update the MODEL_PATH variable.")

# Function to run benchmark of different configurations
def run_benchmark():
    print("\n===== Running Benchmark =====")

    configs = [
        {"name": "Baseline", "threading": False, "quantization": False},
        {"name": "Threading Only", "threading": True, "quantization": False},
        {"name": "Quantization Only", "threading": False, "quantization": True},
        {"name": "Full Optimization", "threading": True, "quantization": True}
    ]

    results = []

    # Process 100 frames with each configuration
    for config in configs:
        print(f"\nTesting: {config['name']}")
        print("-" * 40)

        start_time = time.time()

        # Run the system with this configuration
        stats = run_traffic_monitoring(
            video_path=VIDEO_PATH,
            model_path=MODEL_PATH,
            output_path=None,  # Don't save output for benchmark
            use_threading=config["threading"],
            use_quantization=config["quantization"],
            max_frames=100,
            show_visualization=False  # Disable visualization for benchmark
        )

        elapsed = time.time() - start_time

        if stats:
            results.append({
                "name": config["name"],
                "fps": stats["avg_fps"],
                "time": elapsed,
                "config": config
            })

    # Display results
    if results:
        print("\n===== Benchmark Results =====")
        print("Configuration | Avg FPS | Total Time (s)")
        print("-" * 50)

        for result in results:
            print(f"{result['name']:<15} | {result['fps']:<7.2f} | {result['time']:<12.2f}")

        # Visualize results
        names = [r["name"] for r in results]
        fps_values = [r["fps"] for r in results]

        plt.figure(figsize=(10, 6))
        plt.bar(names, fps_values, color='skyblue')
        plt.title('Performance Comparison (FPS)')
        plt.ylabel('Frames Per Second')
        plt.axhline(y=30, color='r', linestyle='--', label='Real-Time (30 FPS)')
        plt.axhline(y=24, color='g', linestyle='--', label='Smooth Video (24 FPS)')
        plt.legend()

        # Add FPS values on top of bars
        for i, v in enumerate(fps_values):
            plt.text(i, v + 0.5, f"{v:.1f}", ha='center')

        plt.tight_layout()
        plt.show()

        # Return the best configuration
        best_result = max(results, key=lambda x: x["fps"])
        print(f"\nBest configuration: {best_result['name']} with {best_result['fps']:.2f} FPS")
        return best_result["config"]

    # Default configuration if benchmark fails
    return {"threading": True, "quantization": True}

# Run the full demo
def run_full_demo(use_benchmark=True):
    print("\n===== Traffic Monitoring System Demo =====")

    if use_benchmark:
        print("Running benchmark to find optimal configuration...")
        best_config = run_benchmark()
        use_threading = best_config["threading"]
        use_quantization = best_config["quantization"]

        print(f"\nUsing optimal configuration: Threading={use_threading}, Quantization={use_quantization}")
    else:
        # Use default configuration
        use_threading = True
        use_quantization = True
        print("Using default configuration (Threading=True, Quantization=True)")

    print("\nRunning full traffic monitoring system...")
    print("-" * 50)

    # Run the system with the chosen configuration
    stats = run_traffic_monitoring(
        video_path=VIDEO_PATH,
        model_path=MODEL_PATH,
        output_path=OUTPUT_PATH,
        use_threading=use_threading,
        use_quantization=use_quantization,
        max_frames=None,  # Process the entire video
        show_visualization=True
    )

    print("\n===== Demo Complete =====")
    if stats:
        print(f"Processed {stats['frames_processed']} frames at {stats['avg_fps']:.2f} FPS")
        print(f"Detected {stats['unique_vehicles']} unique vehicles")
        print(f"Detected {stats['violations']} traffic violations")

    if os.path.exists(OUTPUT_PATH):
        print(f"Output video saved to: {OUTPUT_PATH}")

# Run the demo
if __name__ == "__main__":
    # Save the system implementation to a Python file
    with open("traffic_monitoring_final.py", "w") as f:
        from IPython import get_ipython
        code = get_ipython().user_ns.get("_")
        if code:
            f.write(code)
        else:
            print("Error: Could not get the code for the traffic monitoring system.")
            print("Please save the code to 'traffic_monitoring_final.py' manually.")

    # Run the demo with benchmark
    run_full_demo(use_benchmark=True)